### Verification notebook

In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [2]:
from sympy import *
from IPython.display import display
import time
import pickle
import datetime
from collections import defaultdict

%env USE_SYMENGINE 1

def Equ(*args, **kwargs):
    kwargs['evaluate'] = False
    return Eq(*args, **kwargs)

env: USE_SYMENGINE=1


In [3]:
#courtesy: https://stackoverflow.com/questions/15463412/differential-operator-usable-in-matrix-form-in-python-module-sympy

from IPython.display import display
from sympy.core.decorators import call_highest_priority
from sympy import Expr, Matrix, Mul, Add, diff, Function
from sympy.core.numbers import Zero
%env USE_SYMENGINE 1

class D(Expr):
    _op_priority = 11.
    is_commutative = False
    diff_symbols = []
    diff_symbols_nc = []
    non_diff_symbols = []
    non_diff_symbols_nc = []

    def __init__(self, *variables, **assumptions):
        super(D, self).__init__()
        self.evaluate = False
        self.variables = variables

    def __repr__(self):
        return 'D%s' % str(self.variables)

    def __str__(self):
        return self.__repr__()

    @call_highest_priority('__mul__')
    def __rmul__(self, other):
        return Mul(other, self)

    @call_highest_priority('__rmul__')
    def __mul__(self, other):
        if isinstance(other, D):
            variables = self.variables + other.variables
            return D(*variables)
        
        if isinstance(other, Matrix):
            other_copy = other.copy()
            for i, elem in enumerate(other):
                other_copy[i] = self * elem
            return other_copy

        if self.evaluate:
            return D.multi_deriv(other, *self.variables)
        else:
            return Mul(self, other)

    def __pow__(self, other):
        variables = self.variables
        for i in range(other-1):
            variables += self.variables
        return D(*variables)

    @staticmethod
    def deriv(expr, xyz):
        if not hasattr(expr, 'free_symbols'):
            return 0
        
        def custom_replace(expr):
            X = Wild('X')
            F = WildFunction('F')
            def replace_helper(F, X):
                if F.args == (X,):
                    return F.func(deriv(X)) / deriv(X)
                else: return Derivative(F, X)        
            return expr.replace(Derivative(F, X), replace_helper)
        # custom_replace makes functions that can not be differentiated commute with differentiaition.
                
        res = 0

        res += custom_replace(Derivative(expr, xyz).doit())

        original = [(sym, sym.is_real) for sym in D.diff_symbols if sym in expr.free_symbols]
        original_nc = [(sym_nc, sym_c.is_real) for sym_c, sym_nc in zip(D.diff_symbols, D.diff_symbols_nc) if sym_nc in expr.free_symbols]

        for sym, is_real in original:
            deriv_term = custom_replace(Derivative(expr, sym).doit())
            if deriv_term != 0:
                newName = multi_var_deriv_name(sym, xyz)

                #dsym = Symbol(newName, commutative=True, real=True)
                dsym = Symbol(newName, commutative=True, real=is_real)
                
                if dsym not in D.diff_symbols:
                    D.create_diff_symbol(newName, real=is_real)
                    
                res += deriv_term * dsym
                
        for sym, is_real in set(original_nc):
            deriv_term = custom_replace(Derivative(expr, sym).doit())
            if deriv_term != 0:
                newName = multi_var_deriv_name(sym, xyz)

                dsym = Symbol(newName, commutative=False)
                
                if dsym not in D.diff_symbols_nc:
                    D.create_diff_symbol(newName, real=is_real)
                    
                res += deriv_term * dsym
                
        return res
    
    @staticmethod
    def multi_deriv(expr, *variables):
        result = expr
        for xyz in variables:
            result = D.deriv(result, xyz)
        return result
    
    @staticmethod
    def create_diff_symbol(name, real=None):
        #new_symbol = Symbol(name, commutative=True, real=True)
        new_symbol = Symbol(name, commutative=True, real=real)
        if new_symbol not in D.diff_symbols:
            D.diff_symbols.append(new_symbol)
        new_symbol_nc = Symbol(name, commutative=False)
        if new_symbol_nc not in D.diff_symbols_nc:
            D.diff_symbols_nc.append(new_symbol_nc)
        return new_symbol, new_symbol_nc

    @staticmethod
    def create_non_diff_symbol(name, real=None):
        # new_symbol = Symbol(name, commutative=True, real=True)
        new_symbol = Symbol(name, commutative=True, real=real)
        if new_symbol not in D.non_diff_symbols:
            D.non_diff_symbols.append(new_symbol)
        new_symbol_nc = Symbol(name, commutative=False)
        if new_symbol_nc not in D.diff_symbols_nc:
            D.non_diff_symbols_nc.append(new_symbol_nc)
        return new_symbol, new_symbol_nc
    
    @staticmethod
    def create_diff_symbols(*names, real=None):
        new_symbols = []
        for name in names:
            new_symbol, new_symbol_nc = D.create_diff_symbol(name, real)
            new_symbols.append(new_symbol_nc)
            new_symbols.append(new_symbol)

        return new_symbols
    
    @staticmethod
    def create_non_diff_symbols(*names, real=None):
        new_symbols = []
        for name in names:
            new_symbol, new_symbol_nc = D.create_non_diff_symbol(name, real)
            new_symbols.append(new_symbol_nc)
            new_symbols.append(new_symbol)
            
        return new_symbols
    
    @staticmethod
    def comm_to_non_comm(expr):
        for (sym, sym_nc) in zip(D.diff_symbols, D.diff_symbols_nc):
            expr = expr.subs(sym, sym_nc)
        for (sym, sym_nc) in zip(D.non_diff_symbols, D.non_diff_symbols_nc):
            expr = expr.subs(sym, sym_nc)
        return expr
    
    @staticmethod
    def non_comm_to_comm(expr):
        for (sym, sym_nc) in zip(D.diff_symbols, D.diff_symbols_nc):
            expr = expr.subs(sym_nc, sym)
        for (sym, sym_nc) in zip(D.non_diff_symbols, D.non_diff_symbols_nc):
            expr = expr.subs(sym_nc, sym)
        return expr
        
    @staticmethod
    def reset_symbols():
        D.diff_symbols = []
        D.diff_symbols_nc = []
        D.non_diff_symbols = []
        D.non_diff_symbols_nc = []
        

def var_deriv_name(var):
    if ')_{x' in var.name:
        return var.name[:-1] + 'x}'
    else:
        return '(' + var.name + ')_{x}'
        #return var.name + '_x'

def multi_var_deriv_name(var, xyz):
    if ')_{' in var.name:
        i = var.name.rindex(')_{') + 3
        derivs = var.name[i:-1]

        return var.name[:i] + ''.join(sorted(derivs + xyz.name)) + '}'
    else:
        return '(' + var.name + ')_{' + xyz.name + '}'
    
def mydiff(expr, *variables):
    if isinstance(expr, D):
        expr.variables += variables
        return D(*expr.variables)
    if isinstance(expr, Matrix):
        expr_copy = expr.copy()
        for i, elem in enumerate(expr):
            expr_copy[i] = D.multi_deriv(expr, *variables)
        return expr_copy
    if isinstance(expr, conjugate):
        return conjugate(D.multi_deriv(expr.args[0], *variables))
    if isinstance(expr, im):
        return im(D.multi_deriv(expr.args[0], *variables))
    if isinstance(expr, re):
        return re(D.multi_deriv(expr.args[0], *variables))
    
    return D.multi_deriv(expr, *variables)


def isFunction(expr):
    return hasattr(expr, 'args') and len(expr.args) > 0 and hasattr(expr, 'func')

spacing = '|   '
def evaluateMul(expr, printing=False, space=spacing, postFunc=None):
    if printing:
        print(space, 'evaluateMul', expr, expr.args)
    if hasattr(expr, 'expand'):
        expr = expr.expand()
    if expr.args:
        if printing:
            print(space, 'hasArgs')
        if isinstance(expr.args[-1], D):
            if printing:
                print(space, 'finalD: zero')
            return Zero()
    initial_args = expr.args
    for i in range(len(expr.args)-1, -1, -1):
        arg = initial_args[i]
        if hasattr(arg, 'expand'):
            arg = arg.expand()
        if printing:
            print(space, 'arg', i, 'is', arg)
        if isinstance(arg, D):
            if printing:
                print(space, 'arg is D')
            left = Mul(*initial_args[:i])
            if printing:
                print(space, 'left', left)
            right = Mul(*expr.args[i+1:])
            if printing:
                print(space, 'right', right)
            right = mydiff(right, *arg.variables)
            if printing:
                print(space, 'new right', right)
            if printing:
                print(space, 'restart')
            return processExpr(left * right, printing=printing, space=space+spacing, postFunc=postFunc)
        else:
            if printing:
                print(space, 'arg is processed further')
            arg = processExpr(arg, printing=printing, space=space+spacing, postFunc=postFunc)
            left = Mul(*initial_args[:i])
            if printing:
                print(space, 'left', left)
            right = Mul(*expr.args[i+1:])
            if printing:
                print(space, 'right', right)
            expr = left * arg * right
            if len(expr.args) < len(initial_args):
                return processExpr(expr, printing=printing, space=space+spacing, postFunc=postFunc)

    if printing:
        print(space, '--Mul-->', expr)
    return postFunc(expr) if postFunc else expr

def processExpr(expr, printing=False, space=spacing, postFunc=None):
    if printing:
        print(space, 'processExpr: ', expr)
    if hasattr(expr, 'expand'):
        expr = expr.expand()
    if isinstance(expr, Matrix):
        if printing:
            print(space, 'Matrix')
        for i, elem in enumerate(expr):
            expr[i] = processExpr(elem, printing=printing, space=space+spacing, postFunc=postFunc)
        if printing:
            print(space, 'newExpr', expr)
    elif isinstance(expr, Mul):
        if printing:
            print(space, 'Mul')
        expr = evaluateMul(expr, printing=printing, space=space+spacing, postFunc=postFunc)
    elif isinstance(expr, D):
        if printing:
            print(space, 'D')
        expr = Zero()
    elif isFunction(expr):
        if printing:
            print(space, 'Function', expr.args)
        new_args = [processExpr(a, printing=printing, space=space+spacing, postFunc=postFunc) for a in expr.args]
        expr = expr.func(*new_args)     
    if printing:
        print(space, '------->', expr)
    return postFunc(expr) if postFunc else expr

def evaluateExpr(expr, printing=False, space=spacing):
    expr = D.comm_to_non_comm(expr)
    expr = processExpr(expr, printing=printing, space=space)
    expr = D.non_comm_to_comm(expr)
    return(expr)

def test_suite():
    D.reset_symbols()
    x, x_c, y, y_c, z, z_c = D.create_non_diff_symbols('x', 'y', 'z')
    qq, qq_c, qq_conj, qq_conj_c, = D.create_diff_symbols('q', '\overline{q}')
    uu, uu_c, ff, ff_c, gg, gg_c, vv, vv_c = D.create_diff_symbols('u', 'f', 'g', 'v')
    AA, AA_c, pphi, pphi_c = D.create_diff_symbols('A', '\phi')

    theta = I / 2 * (pphi - pi / 2)

    M = 1 / sqrt(2) * Matrix([[exp(- theta), exp(theta)], [exp(- theta), - exp(theta)]])
    M_conj = 1 / sqrt(2) * Matrix([[exp(theta), exp(- theta)], [exp(theta), - exp(- theta)]])
    
    Lax = I * Matrix([[D(x), - qq], [qq_conj, - D(x)]]).subs(qq, AA * exp(I * pphi)).subs(qq_conj, AA * exp(- I * pphi))
    
    Pax = I * Matrix([[2 * D(x, x) - (AA**2 - 1), - AA * exp(I * pphi) * D(x) - D(x) * AA * exp(I * pphi)],
                    [AA * exp(- I * pphi) * D(x) + D(x) * AA * exp(- I * pphi), - 2 * D(x, x) + (AA**2 - 1)]])
    
    F = Matrix([[ff, 0], [0, gg]])
    
    res1 = evaluateExpr(Matrix([[AA - (D(x) * pphi - pphi * D(x)) / 2, I * D(x)], 
                   [I * D(x), - AA - (D(x) * pphi - pphi * D(x)) / 2]]) * F)
    
    res2 = evaluateExpr(evaluateExpr(Matrix([[0, (D(x, x) - 2 * (AA**2 - 1)) * AA * exp(I * pphi)],
                [(D(x, x) - 2 * (AA**2 - 1)) * AA * exp(- I * pphi), 0]])) * F)

    
    old_expr = M * Lax * Transpose(M_conj) * F
    expr = old_expr.copy()
    new_expr = evaluateExpr(expr)
    display(simplify(new_expr))
    if simplify(new_expr - res1) == Matrix([[0, 0], [0, 0]]):
        print('Test 1: success')
    else:
        print('Test 2: failed')
    old_expr = (Pax * Lax - Lax * Pax) * F
    expr = old_expr.copy()
    new_expr = evaluateExpr(expr)
    display(simplify(new_expr))
    if simplify(new_expr - res2) == Matrix([[0, 0], [0, 0]]):
        print('Test 2: success')
    else:
        print('Test 2: failed')
    D.reset_symbols()
    
test_suite()

env: USE_SYMENGINE=1


Matrix([
[f*(-(\phi)_{x} + 2*A)/2,               I*(g)_{x}],
[              I*(f)_{x}, g*(-(\phi)_{x} - 2*A)/2]])

Test 1: success


Matrix([
[                                                                                                    0, g*((A)_{xx} + 2*I*(A)_{x}*(\phi)_{x} + I*(\phi)_{xx}*A - (\phi)_{x}**2*A - 2*A**3 + 2*A)*exp(I*\phi)],
[f*((A)_{xx} - 2*I*(A)_{x}*(\phi)_{x} - I*(\phi)_{xx}*A - (\phi)_{x}**2*A - 2*A**3 + 2*A)*exp(-I*\phi),                                                                                                    0]])

Test 2: success


In [4]:
from collections import deque
from multiset import Multiset

def get_var_name_from_deriv(sym):
    start = sym.name.find('(')
    end = sym.name.find(')')
    if start != -1 and end != -1:
        sym_name = sym.name[start+1:end]
        return sym_name
    else:
        sym_name = sym.name
        return sym_name
        
def get_multiindex_from_deriv(sym):
    start = sym.name.find('(')
    end = sym.name.find(')')
    if start != -1 and end != -1:
        return sym.name[end+3:-1]
    else:
        return ''
    
def get_order_from_deriv(sym):
    start = sym.name.find('(')
    end = sym.name.find(')')
    if start != -1 and end != -1:
        sym_name = sym.name[start+1:end]
        return len(sym.name) - 4 - sym.name.find(')_{')
    else:
        return 0

def deriv(poly):
    return multi_deriv(poly, [x])

def higher_deriv(poly, n):
    return multi_deriv(poly, [x] * n)
   
def multi_deriv(expr, xyz):
    if isinstance(xyz, (list, tuple)):
        result = expr
        for elem in xyz:
            result = multi_deriv(result, elem)
        return result
    
    if isinstance(expr, Matrix):
        expr_copy = expr.copy()
        for i, elem in enumerate(expr):
            expr_copy[i] = multi_deriv(elem, xyz)
        return expr_copy
    
    if not hasattr(expr, 'free_symbols'):
        return 0
    
    res = 0
    res += Derivative(expr, xyz).doit()

    fixed_symbols = (set(D.diff_symbols) | set(D.diff_symbols_nc)) & set(expr.free_symbols)
    symbols_to_iterate = list(fixed_symbols)  # Creates a separate list copy

    for sym in symbols_to_iterate:
        
        def custom_replace(expr):
            X = Wild('X')
            F = WildFunction('F')
            def replace_helper(F, X):
                if F.args == (X,):
                    return F.func(deriv(X)) / deriv(X)
                else: return Derivative(F, X)        
            return expr.replace(Derivative(F, X), replace_helper)
        # custom_replace makes functions that can not be differentiated commute with differentiaition.
        deriv_term = custom_replace(Derivative(expr, sym).doit())
        if deriv_term != 0:
            newName = multi_var_deriv_name(sym, xyz)
            if sym.is_commutative:
                dsym = next((s for s in D.diff_symbols if s.name == newName), D.create_diff_symbols(newName, real=sym.is_real)[1])
            else:
                dsym = next((s for s in D.diff_symbols_nc if s.name == newName), D.create_diff_symbols(newName, real=sym.is_real)[0])
            res += deriv_term * dsym
    
    return res

def single_subs(expr, var, sub, scale=1):
    if not hasattr(expr, 'subs'):
        return expr
        
    if isinstance(scale, dict):
        for xyz, scaling in scale.items():
            expr = expr.subs(xyz, scaling * xyz)
            
    if var not in set(D.diff_symbols) | set(D.diff_symbols_nc):
        return expr.subs(var, sub)
    
    var_name = get_var_name_from_deriv(var)
    var_multiindex = Multiset(get_multiindex_from_deriv(var))
    
    fixed_symbols = (set(D.diff_symbols) | set(D.diff_symbols_nc)) & set(expr.free_symbols)
    symbols_to_iterate = list(fixed_symbols)  # Creates a separate list copy

    for sym in symbols_to_iterate:
        if get_var_name_from_deriv(sym) == var_name:
            sym_multiindex = Multiset(get_multiindex_from_deriv(sym))
            if var_multiindex.issubset(sym_multiindex):
                target_multiindex = sym_multiindex.difference(var_multiindex)
                target_operator = [Symbol(char, real=True) for char in target_multiindex]
                #target_operator = [Symbol(char) for char in target_multiindex]
                if isinstance(scale, dict):
                    factor = 1
                    for direction in target_operator:
                        factor *= scale.get(direction, 1)
                else:
                    factor = scale**len(target_operator)
                expr = expr.subs(sym, factor * multi_deriv(sub, target_operator))

    return expr

def subs(expr, data, scale=1):
    for (var, sub) in data:
        expr = single_subs(expr, var, sub, scale=scale)
    return expr

def variation(expr, sym):
    if not hasattr(expr, 'free_symbols'):
        return 0
    
    res = 0
    order = 0

    syms = []
    orders = []

    start = sym.name.find('(')
    end = sym.name.find(')')
    if start != -1 and end != -1:
        sym_name = sym.name[start+1:end]
        syms.append(sym)
        orders.append(len(sym.name) - 4 - sym.name.find(')_{'))
    else:
        sym_name = sym.name
        syms.append(sym)
        orders.append(0)

    for s in expr.free_symbols:
        start = s.name.find('(')
        end = s.name.find(')')
        if start != -1 and end != -1 and sym_name == s.name[start+1:end]:
            if s.name not in [sym.name for sym in syms]:
                syms.append(s)
                orders.append(len(s.name) - 4 - s.name.find(')_{'))    
    
    for (sym, order) in zip(syms, orders):
        res += (-1)**order * higher_deriv(Derivative(expr, sym).doit(), order)

    return simplify(res)

epsilon, epsiloninv = symbols('\epsilon, \eta', positive=True)

def polynomize(expr):
    return simplify(Poly(expr, epsilon, epsiloninv).subs(epsiloninv, 1/epsilon))

def depolynomize(poly):
    monoms = poly.monoms()
    coeffs = poly.coeffs()
    
    X = 0
    for (k, m) in enumerate(monoms):
        X += epsilon**m[0] * epsiloninv**m[1] * coeffs[k]
            
    return X

def poly_simplify(expr):
    return depolynomize(polynomize(expr))


def traverse(expr):
    if any(a.is_Number and abs(a) != 1 for a in expr.args):
        print(expr,'->',expr.func(*[(a if not a.is_Number else 1) for a in expr.args]))
    if expr.is_Number and abs(expr) != 1:
        return 1, True
    oldexpr = expr
    args = expr.args
    sargs = []
    hit = False
    for arg in args:
        arg,arghit = traverse(arg)
        hit |= arghit
        sargs.append(arg)

    if(hit):
        expr = expr.func(*sargs)
        return expr, True
    else:
        return oldexpr, False

def extract_deriv(expr, k): 
    expr = simplify(expr).expand()
    #print("expr =", expr) 
    res = 0
    if isinstance(expr, Add):
        for mon in expr.args:
            #print("mon =", mon)
            count = 0
            if hasattr(mon, "name"):
                #print(mon.name)
                if "_{x" in mon.name:
                    count += 1
                    #print("count =", count)
            else:
                for fac in mon.args:
                    #print("fac =", fac)
                    if hasattr(fac, "name"):
                        #print(fac.name)
                        if "_{x" in fac.name:
                            count += 1
                            #print("count =", count)
                    elif isinstance(fac, Pow):
                        #print(fac.base.name, fac.exp)
                        if "_{x" in fac.base.name and fac.exp > 0:
                            count += fac.exp     
                            #print("count =", count)
            #print("final count =", count)
            if count == k:
                res += mon

    else:
        mon = expr
        #print("mon =", mon)
        count = 0
        if hasattr(mon, "name"):
            #print(mon.name)
            if "_{x" in mon.name:
                count += 1
                #print("count =", count)
        elif hasattr(mon, "args"):
            for fac in mon.args:
                #print("fac =", fac)
                if hasattr(fac, "name"):
                    #print(fac.name)
                    if "_{x" in fac.name:
                        count += 1
                        #print("count =", count)
                elif isinstance(fac, Pow):
                    #print(fac.base.name, fac.exp)
                    if "_{x" in fac.base.name and fac.exp > 0:
                        count += fac.exp     
                        #print("count =", count)
        #print("final count =", count)
        if count == k:
            res += mon

    #print("res =", res)
    return res
        
def extract_deriv_alt(expr, k):
    expr = simplify(expr).expand()
    #print("expr =", expr) 
    res = 0
    if isinstance(expr, Add):
        for mon in expr.args:
            presence_deriv_q_conj = False
            #print("mon =", mon)
            count = 0
            if hasattr(mon, "name"):
                #print(mon.name)
                if "_{x" in mon.name:
                    count += 1
                    if get_var_name_from_deriv(mon) == q_conj.name:
                        presence_deriv_q_conj = True
            else:
                for fac in mon.args:
                    #print("fac =", fac)
                    if hasattr(fac, "name"):
                        #print(fac.name)
                        if "_{x" in fac.name:
                            count += 1
                            if get_var_name_from_deriv(fac) == q_conj.name:
                                presence_deriv_q_conj = True
                            #print("count =", count)
                    elif isinstance(fac, Pow):
                        #print(fac.base.name, fac.exp)
                        if "_{x" in fac.base.name and fac.exp > 0:
                            count += fac.exp     
                            if get_var_name_from_deriv(fac.base) == q_conj.name:
                                presence_deriv_q_conj = True
                            #print("count =", count)
            #print("final count =", count)
            if count == k and presence_deriv_q_conj:
                res += mon

    else:
        presence_deriv_q_conj = False
        mon = expr
        #print("mon =", mon)
        count = 0
        if hasattr(mon, "name"):
            #print(mon.name)
            if "_{x" in mon.name:
                count += 1
                if get_var_name_from_deriv(mon) == q_conj.name:
                    presence_deriv_q_conj = True
                #print("count =", count)
        elif hasattr(mon, "args"):
            for fac in mon.args:
                #print("fac =", fac)
                if hasattr(fac, "name"):
                    #print(fac.name)
                    if "_{x" in fac.name:
                        count += 1
                        if get_var_name_from_deriv(fac) == q_conj.name:
                            presence_deriv_q_conj = True
                        #print("count =", count)
                elif isinstance(fac, Pow):
                    #print(fac.base.name, fac.exp)
                    if "_{x" in fac.base.name and fac.exp > 0:
                        count += fac.exp   
                        if get_var_name_from_deriv(fac.base) == q_conj.name:
                            presence_deriv_q_conj = True
                        #print("count =", count)
        #print("final count =", count)
        if count == k and presence_deriv_q_conj:
            res += mon

    #print("res =", res)
    return res



# Symbols

In [5]:
D.reset_symbols()
xx, x, yy, y, tt, t = D.create_non_diff_symbols('x', 'y', 't', real=True)
qq, q, qq_conj, q_conj = D.create_diff_symbols('q', '\\tilde{q}')
pp, p, pp_conj, p_conj = D.create_diff_symbols('p', '\\tilde{p}')
AA, A, aa, a = D.create_diff_symbols('A', 'a', real=True)
uu, u, vv, v, aa, a, pphi, phi, ff, f, gg, g = D.create_diff_symbols('u', 'v', 'a', '\\phi', 'f', 'g', real=True)
WW_plus, W_plus, WW_minus, W_minus, ww_plus, w_plus, ww_minus, w_minus = D.create_diff_symbols('W_+', 'W_-', 'w_+', 'w_-', real=True)
qq_pm, q_pm, qq_conj_pm, q_conj_pm = D.create_non_diff_symbols('q_\\pm', '\\tilde{q}_\pm')
pphi_pm, phi_pm, pphi_minus, phi_minus, pphi_plus, phi_plus = D.create_non_diff_symbols('\\phi_\pm', '\\phi_-', '\\phi_+', real=True)
ssig, sig, rr, r, ssigt, sigt, rrt, rt = D.create_diff_symbols('\\sigma', 'r', '\\tilde{\\sigma}', '\\tilde{r}')
kap = Symbol('\\kappa', positive=True)
lam, z = symbols('\\lambda z', real=False)
tlam, tz = symbols('\\tilde{\\lambda} \\tilde{z}', positive=True)

PPhi_11, Phi_11, PPhi_12, Phi_12, PPhi_21, Phi_21, PPhi_22, Phi_22 = D.create_diff_symbols(
    '\\Phi_{1,1}', '\\Phi_{1,2}', '\\Phi_{2,1}', '\\Phi_{2,2}'
)
PPhi_plus_11, Phi_plus_11, PPhi_plus_12, Phi_plus_12, PPhi_plus_21, Phi_plus_21, PPhi_plus_22, Phi_plus_22 = D.create_diff_symbols(
    '\\Phi_{1,1}^+', '\\Phi_{1,2}^+', '\\Phi_{2,1}^+', '\\Phi_{2,2}^+'
)
PPhi_minus_11, Phi_minus_11, PPhi_minus_12, Phi_minus_12, PPhi_minus_21, Phi_minus_21, PPhi_minus_22, Phi_minus_22 = D.create_diff_symbols(
    '\\Phi_{1,1}^-', '\\Phi_{1,2}^-', '\\Phi_{2,1}^-', '\\Phi_{2,2}^-'
)
PPsi_11, Psi_11, PPsi_12, Psi_12, PPsi_21, Psi_21, PPsi_22, Psi_22 = D.create_diff_symbols(
    '\\Psi_{1,1}', '\\Psi_{1,2}', '\\Psi_{2,1}', '\\Psi_{2,2}'
)
PPsi_plus_11, Psi_plus_11, PPsi_plus_12, Psi_plus_12, PPsi_plus_21, Psi_plus_21, PPsi_plus_22, Psi_plus_22 = D.create_diff_symbols(
    '\\Psi_{1,1}^+', '\\Psi_{1,2}^+', '\\Psi_{2,1}^+', '\\Psi_{2,2}^+'
)
PPsi_minus_11, Psi_minus_11, PPsi_minus_12, Psi_minus_12, PPsi_minus_21, Psi_minus_21, PPsi_minus_22, Psi_minus_22 = D.create_diff_symbols(
    '\\Psi_{1,1}^-', '\\Psi_{1,2}^-', '\\Psi_{2,1}^-', '\\Psi_{2,2}^-'
)
PPhi = Matrix([[PPhi_11, PPhi_21],
              [PPhi_12, PPhi_22]])
Phi = Matrix([[Phi_11, Phi_21],
             [Phi_12, Phi_22]])
PPsi = Matrix([[PPsi_11, PPsi_21],
              [PPsi_12, PPsi_22]])
Psi = Matrix([[Psi_11, Psi_21],
             [Psi_12, Psi_22]])
PPhi_plus = Matrix([[PPhi_plus_11, PPhi_plus_21],
                   [PPhi_plus_12, PPhi_plus_22]])
Phi_plus = Matrix([[Phi_plus_11, Phi_plus_21],
                  [Phi_plus_12, Phi_plus_22]])
PPhi_minus = Matrix([[PPhi_minus_11, PPhi_minus_21],
                    [PPhi_minus_12, PPhi_minus_22]])
Phi_minus = Matrix([[Phi_minus_11, Phi_minus_21],
                   [Phi_minus_12, Phi_minus_22]])
PPsi_plus = Matrix([[PPsi_plus_11, PPsi_plus_21],
                   [PPsi_plus_12, PPsi_plus_22]])
Psi_plus = Matrix([[Psi_plus_11, Psi_plus_21],
                  [Psi_plus_12, Psi_plus_22]])
PPsi_minus = Matrix([[PPsi_minus_11, PPsi_minus_21],
                    [PPsi_minus_12, PPsi_minus_22]])
Psi_minus = Matrix([[Psi_minus_11, Psi_minus_21],
                   [Psi_minus_12, Psi_minus_22]])


def tsimplify(expr):
    return simplify(simplify(expr.subs(lam, I * tlam).subs(z, I * tz)).subs(tlam, lam / I).subs(tz, z / I))
def realify(expr):
    return expr.subs(lam, I * tlam).subs(z, I * tz)
def complexify(expr):
    return expr.subs(tz, - I * z).subs(tlam, - I * lam)



# Equations

In [6]:
KdV_LHS = multi_deriv(u, t)
KdV_RHS = 6 * u * deriv(u) - higher_deriv(u, 3)

NLS_q_LHS = I * multi_deriv(q, t) 
NLS_q_RHS = - higher_deriv(q, 2) + 2 * q**2 * q_conj

NLS_q_conj_LHS = I * multi_deriv(q_conj, t) 
NLS_q_conj_RHS = higher_deriv(q_conj, 2) - 2 * q_conj**2 * q

GP_q_LHS = I * multi_deriv(q, t) 
GP_q_RHS = - higher_deriv(q, 2) + 2 * q * (q * q_conj - 1)

GP_q_conj_LHS = I * multi_deriv(q_conj, t) 
GP_q_conj_RHS = higher_deriv(q_conj, 2) - 2 * q_conj * (q_conj * q - 1)

hGP_a_LHS = multi_deriv(a, t)
hGP_a_RHS = - a * deriv(v) - 2 * v * deriv(a)
hGP_v_LHS = multi_deriv(v, t)
hGP_v_RHS = - deriv(v**2) - 2 * deriv(a**2) + deriv(higher_deriv(a, 2) / a)

hGP_w_minus_LHS = multi_deriv(w_minus, t)
hGP_w_minus_RHS = - (3 * w_minus + w_plus - 2) * deriv(w_minus) + Rational(1,2) * deriv(higher_deriv(w_plus - w_minus, 2) / (w_plus - w_minus + 2))
hGP_w_plus_LHS = multi_deriv(w_plus, t)
hGP_w_plus_RHS = - (w_minus + 3 * w_plus + 2) * deriv(w_plus) + Rational(1,2) * deriv(higher_deriv(w_plus - w_minus, 2) / (w_plus - w_minus + 2))

rGP_W_minus_LHS = multi_deriv(W_minus, t)
rGP_W_minus_RHS = - (6 * W_minus + 2 * W_plus - 4 / epsilon**2) * deriv(W_minus) + 2 * deriv(higher_deriv(W_plus - W_minus, 2) / (epsilon**2 * W_plus - epsilon**2 * W_minus + 2))
rGP_W_plus_LHS = multi_deriv(W_plus, t)
rGP_W_plus_RHS = - (2 * W_minus + 6 * W_plus + 4 / epsilon**2) * deriv(W_plus) + 2 * deriv(higher_deriv(W_plus - W_minus, 2) / (epsilon**2 * W_plus - epsilon**2 * W_minus + 2))


display(Eq(KdV_LHS, KdV_RHS, evaluate=False))
display(Eq(NLS_q_LHS, NLS_q_RHS, evaluate=False))
display(Eq(NLS_q_conj_LHS, NLS_q_conj_RHS, evaluate=False))
display(Eq(GP_q_LHS, GP_q_RHS, evaluate=False))
display(Eq(GP_q_conj_LHS, GP_q_conj_RHS, evaluate=False))
display(Eq(hGP_a_LHS, hGP_a_RHS, evaluate=False))
display(Eq(hGP_v_LHS, hGP_v_RHS, evaluate=False))
display(Eq(hGP_w_minus_LHS, hGP_w_minus_RHS, evaluate=False))
display(Eq(hGP_w_plus_LHS, hGP_w_plus_RHS, evaluate=False))
display(Eq(rGP_W_minus_LHS, rGP_W_minus_RHS, evaluate=False))
display(Eq(rGP_W_plus_LHS, rGP_W_plus_RHS, evaluate=False))


substituted_hGP_a_LHS = subs(hGP_a_LHS, [(a, sqrt(q * q_conj)), (v, (q_conj * deriv(q) - q * deriv(q_conj)) / (2 * I * q * q_conj))])
substituted_hGP_a_RHS = subs(hGP_a_RHS, [(a, sqrt(q * q_conj)), (v, (q_conj * deriv(q) - q * deriv(q_conj)) / (2 * I * q * q_conj))])
substituted_hGP_v_LHS = subs(hGP_v_LHS, [(a, sqrt(q * q_conj)), (v, (q_conj * deriv(q) - q * deriv(q_conj)) / (2 * I * q * q_conj))])
substituted_hGP_v_RHS = subs(hGP_v_RHS, [(a, sqrt(q * q_conj)), (v, (q_conj * deriv(q) - q * deriv(q_conj)) / (2 * I * q * q_conj))])

equation_applied_to_substituted_hGP_a_LHS = subs(substituted_hGP_a_LHS, [(simplify(GP_q_LHS / I), GP_q_RHS / I), (simplify(GP_q_conj_LHS / I), GP_q_conj_RHS / I)])
equation_applied_to_substituted_hGP_v_LHS = subs(substituted_hGP_v_LHS, [(simplify(GP_q_LHS / I), GP_q_RHS / I), (simplify(GP_q_conj_LHS / I), GP_q_conj_RHS / I)])

display(Equ(Symbol('\\text{Error}_{(q, \overline{q}) \leftrightarrow (a, v)}^1'), simplify(equation_applied_to_substituted_hGP_a_LHS - substituted_hGP_a_RHS)))
display(Equ(Symbol('\\text{Error}_{(q, \overline{q}) \leftrightarrow (a, v)}^2'), simplify(equation_applied_to_substituted_hGP_v_LHS - substituted_hGP_v_RHS)))


substituted_hGP_w_minus_LHS = subs(hGP_w_minus_LHS, [(w_minus, v / 2 - (a - 1)), (w_plus, v / 2 + (a - 1))])
substituted_hGP_w_minus_RHS = subs(hGP_w_minus_RHS, [(w_minus, v / 2 - (a - 1)), (w_plus, v / 2 + (a - 1))])
substituted_hGP_w_plus_LHS = subs(hGP_w_plus_LHS, [(w_minus, v / 2 - (a - 1)), (w_plus, v / 2 + (a - 1))])
substituted_hGP_w_plus_RHS = subs(hGP_w_plus_RHS, [(w_minus, v / 2 - (a - 1)), (w_plus, v / 2 + (a - 1))])

equation_applied_to_substituted_hGP_w_minus_LHS = subs(substituted_hGP_w_minus_LHS, [(hGP_a_LHS, hGP_a_RHS), (hGP_v_LHS, hGP_v_RHS)])
equation_applied_to_substituted_hGP_w_plus_LHS = subs(substituted_hGP_w_plus_LHS, [(hGP_a_LHS, hGP_a_RHS), (hGP_v_LHS, hGP_v_RHS)])

display(Equ(Symbol('\\text{Error}_{(a, v) \leftrightarrow (w_-, w_+)}^1'), simplify(equation_applied_to_substituted_hGP_w_minus_LHS - substituted_hGP_w_minus_RHS)))
display(Equ(Symbol('\\text{Error}_{(a, v) \leftrightarrow (w_-, w_+)}^2'), simplify(equation_applied_to_substituted_hGP_w_plus_LHS - substituted_hGP_w_plus_RHS)))


substituted_rGP_W_minus_LHS = subs(rGP_W_minus_LHS, [(W_minus, epsilon**(-2) * w_minus), (W_plus, epsilon**(-2) * w_plus)], scale={t: epsilon**(-3) * sqrt(2), x: epsilon**(-1) / sqrt(2)})
substituted_rGP_W_minus_RHS = subs(rGP_W_minus_RHS, [(W_minus, epsilon**(-2) * w_minus), (W_plus, epsilon**(-2) * w_plus)], scale={t: epsilon**(-3) * sqrt(2), x: epsilon**(-1) / sqrt(2)})
substituted_rGP_W_plus_LHS = subs(rGP_W_plus_LHS, [(W_minus, epsilon**(-2) * w_minus), (W_plus, epsilon**(-2) * w_plus)], scale={t: epsilon**(-3) * sqrt(2), x: epsilon**(-1) / sqrt(2)})
substituted_rGP_W_plus_RHS = subs(rGP_W_plus_RHS, [(W_minus, epsilon**(-2) * w_minus), (W_plus, epsilon**(-2) * w_plus)], scale={t: epsilon**(-3) * sqrt(2), x: epsilon**(-1) / sqrt(2)})

equation_applied_to_substituted_rGP_W_minus_LHS = subs(substituted_rGP_W_minus_LHS, [(hGP_w_minus_LHS, hGP_w_minus_RHS), (hGP_w_plus_LHS, hGP_w_plus_RHS)], scale={t: epsilon**(-3) * sqrt(2), x: epsilon**(-1) / sqrt(2)})
equation_applied_to_substituted_rGP_W_plus_LHS = subs(substituted_rGP_W_plus_LHS, [(hGP_w_minus_LHS, hGP_w_minus_RHS), (hGP_w_plus_LHS, hGP_w_plus_RHS)], scale={t: epsilon**(-3) * sqrt(2), x: epsilon**(-1) / sqrt(2)})

display(Equ(Symbol('\\text{Error}_{(w_-, w_+) \leftrightarrow (W_-, W_+)}^2'), simplify(equation_applied_to_substituted_rGP_W_minus_LHS - substituted_rGP_W_minus_RHS)))
display(Equ(Symbol('\\text{Error}_{(w_-, w_+) \leftrightarrow (W_-, W_+)}^2'), simplify(equation_applied_to_substituted_rGP_W_plus_LHS - substituted_rGP_W_plus_RHS)))



Eq((u)_{t}, -(u)_{xxx} + 6*(u)_{x}*u)

Eq(I*(q)_{t}, -(q)_{xx} + 2*\tilde{q}*q**2)

Eq(I*(\tilde{q})_{t}, (\tilde{q})_{xx} - 2*\tilde{q}**2*q)

Eq(I*(q)_{t}, -(q)_{xx} + 2*q*(\tilde{q}*q - 1))

Eq(I*(\tilde{q})_{t}, (\tilde{q})_{xx} - 2*\tilde{q}*(\tilde{q}*q - 1))

...

Eq(\text{Error}_{(q, \overline{q}) \leftrightarrow (a, v)}^2, 0)

Eq(\text{Error}_{(a, v) \leftrightarrow (w_-, w_+)}^1, 0)

Eq(\text{Error}_{(a, v) \leftrightarrow (w_-, w_+)}^2, 0)

Eq(\text{Error}_{(w_-, w_+) \leftrightarrow (W_-, W_+)}^2, 0)

Eq(\text{Error}_{(w_-, w_+) \leftrightarrow (W_-, W_+)}^2, 0)

# Equations vs Lax Pairs

In [7]:
L_KdV = - D(xx, xx) + uu
P_KdV = - 4 * D(xx, xx, xx) + 3 * evaluateExpr(D(xx) * uu) + 6 * uu * D(xx)

tested_KdV_Lax_equation = evaluateExpr((evaluateExpr(D(tt) * L_KdV) + L_KdV * P_KdV - P_KdV * L_KdV) * ff)
tested_KdV_equation = (KdV_LHS - KdV_RHS) * f

display(Equ(Symbol('\\text{Error}_{\\text{KdV vs Lax pair}}'), simplify(tested_KdV_Lax_equation - tested_KdV_equation)))


L_NLS = I * Matrix([[D(xx), - qq], [qq_conj, - D(xx)]])
P_NLS = I * Matrix([[2 * D(xx, xx) - qq * qq_conj, - qq * D(xx) - D(xx) * qq], [qq_conj * D(xx) + D(xx) * qq_conj, - 2 * D(xx, xx) + qq * qq_conj]])

tested_NLS_Lax_equation = evaluateExpr((processExpr(D(tt) * L_NLS) + L_NLS * P_NLS - P_NLS * L_NLS) * ff)
tested_NLS_equation = Matrix([[0, NLS_q_RHS - NLS_q_LHS], [- NLS_q_conj_RHS + NLS_q_conj_LHS, 0]]) * f

display(Equ(Symbol('\\text{Error}_{\\text{NLS vs Lax pair}}'), simplify(tested_NLS_Lax_equation - tested_NLS_equation)))


L_GP = I * Matrix([[D(xx), - qq], [qq_conj, - D(xx)]])
P_GP = I * Matrix([[2 * D(xx, xx) - (qq * qq_conj - 1), - qq * D(xx) - D(xx) * qq], [qq_conj * D(xx) + D(xx) * qq_conj, - 2 * D(xx, xx) + (qq * qq_conj - 1)]])

tested_GP_Lax_equation = evaluateExpr((processExpr(D(tt) * L_GP) + L_GP * P_GP - P_GP * L_GP) * ff)
tested_GP_equation = Matrix([[0, GP_q_RHS - GP_q_LHS], [- GP_q_conj_RHS + GP_q_conj_LHS, 0]]) * f

display(Equ(Symbol('\\text{Error}_{\\text{GP vs Lax pair}}'), simplify(tested_GP_Lax_equation - tested_GP_equation)))


L_hGP_av = Matrix([[aa - vv / 2, I * D(xx)], [I * D(xx), - aa - vv / 2]])
P_hGP_av = Matrix([
    [processExpr(D(xx) * (aa - vv)) + 2 * (aa - vv) * D(xx),
     I / 2 * (4 * D(xx, xx) - processExpr(D(xx, xx) * aa) / aa)], 
    [I / 2 * (4 * D(xx, xx) - processExpr(D(xx, xx) * aa) / aa),
     processExpr(D(xx) * (- aa - vv)) + 2 * (- aa - vv) * D(xx)
]])

tested_hGP_av_Lax_equation = evaluateExpr((processExpr(D(tt) * L_hGP_av) + L_hGP_av * P_hGP_av - P_hGP_av * L_hGP_av) * ff)
tested_hGP_av_equation = Matrix([[(hGP_v_RHS - hGP_v_LHS) / 2 - (hGP_a_RHS - hGP_a_LHS), 0], 
                                 [0, (hGP_v_RHS - hGP_v_LHS) / 2 + (hGP_a_RHS - hGP_a_LHS)]]) * f

display(Equ(Symbol('\\text{Error}_{\\text{hGP vs Lax pair}}^{(a, v)}'), simplify(tested_hGP_av_Lax_equation - tested_hGP_av_equation)))


L_hGP_w = Matrix([[- ww_minus + 1, I * D(xx)], [I * D(xx), - ww_plus - 1]])
P_hGP_w = Matrix([
    [processExpr(- D(xx) * (3 * ww_minus + ww_plus - 2) / 2) - (3 * ww_minus + ww_plus - 2) * D(xx),
     I / 2 * (4 * D(xx, xx) - processExpr(D(xx, xx) * (ww_plus - ww_minus + 2) / 2) / ((ww_plus - ww_minus + 2) / 2))], 
    [I / 2 * (4 * D(xx, xx) - processExpr(D(xx, xx) * (ww_plus - ww_minus + 2) / 2) / ((ww_plus - ww_minus + 2) / 2)),
     processExpr(- D(xx) * (ww_minus + 3 * ww_plus + 2) / 2) - (ww_minus + 3 * ww_plus + 2) * D(xx)]
])

tested_hGP_w_Lax_equation = evaluateExpr((processExpr(D(tt) * L_hGP_w) + L_hGP_w * P_hGP_w - P_hGP_w * L_hGP_w) * ff)
tested_hGP_w_equation = Matrix([[hGP_w_minus_RHS - hGP_w_minus_LHS, 0], [0, hGP_w_plus_RHS - hGP_w_plus_LHS]]) * f

display(Equ(Symbol('\\text{Error}_{\\text{hGP vs Lax pair}}^{(w_-, w_+)}'), simplify(tested_hGP_w_Lax_equation - tested_hGP_w_equation)))


L_rGP_W = Matrix([[- WW_minus + 1 / epsilon**2, I * sqrt(2) / epsilon * D(xx)], [I * sqrt(2) / epsilon * D(xx), - WW_plus - 1 / epsilon**2]])
P_rGP_W = sqrt(2) / epsilon**3 * Matrix([
    [processExpr(- sqrt(2) * epsilon * D(xx) * (3 * epsilon**2 * WW_minus + epsilon**2 * WW_plus - 2) / 2) - (3 * epsilon**2 * WW_minus + epsilon**2 * WW_plus - 2) * sqrt(2) * epsilon * D(xx),
     I / 2 * (4 * 2 * epsilon**2 * D(xx, xx) - processExpr(2 * epsilon**2 * D(xx, xx) * (epsilon**2 * WW_plus - epsilon**2 * WW_minus + 2) / 2) / ((epsilon**2 * WW_plus - epsilon**2 * WW_minus + 2) / 2))], 
    [I / 2* (4  * 2 * epsilon**2 * D(xx, xx) - processExpr(2 * epsilon**2 * D(xx, xx) * (epsilon**2 * WW_plus - epsilon**2 * WW_minus + 2) / 2) / ((epsilon**2 * WW_plus - epsilon**2 * WW_minus + 2) / 2)),
     processExpr(- sqrt(2) * epsilon * D(xx) * (epsilon**2 * WW_minus + 3 * epsilon**2 * WW_plus + 2) / 2) - (epsilon**2 * WW_minus + 3 * epsilon**2 * WW_plus + 2) * sqrt(2) * epsilon * D(xx)]
])

tested_rGP_W_Lax_equation = evaluateExpr((processExpr(D(tt) * L_rGP_W) + L_rGP_W * P_rGP_W - P_rGP_W * L_rGP_W) * ff)
tested_rGP_W_equation = Matrix([[rGP_W_minus_RHS - rGP_W_minus_LHS, 0], [0, rGP_W_plus_RHS - rGP_W_plus_LHS]]) * f

display(Equ(Symbol('\\text{Error}_{\\text{rGP vs Lax pair}}^{(W_-, W_+)}'), simplify(tested_rGP_W_Lax_equation - tested_rGP_W_equation)))


theta = I / 2 * (phi - pi / 2)
ttheta = I / 2 * (pphi - pi / 2)
theta_pm = I / 2 * (phi_pm - pi / 2)
ttheta_pm = I / 2 * (pphi_pm - pi / 2)
M = 1 / sqrt(2) * Matrix([[exp(- theta), exp(theta)], [exp(- theta), - exp(theta)]])
MM = 1 / sqrt(2) * Matrix([[exp(- ttheta), exp(ttheta)], [exp(- ttheta), - exp(ttheta)]])
M_conj = 1 / sqrt(2) * Matrix([[exp(theta), exp(- theta)], [exp(theta), - exp(- theta)]])
MM_conj = 1 / sqrt(2) * Matrix([[exp(ttheta), exp(- ttheta)], [exp(ttheta), - exp(- ttheta)]])
M_pm = 1 / sqrt(2) * Matrix([[exp(- theta_pm), exp(theta_pm)], [exp(- theta_pm), - exp(theta_pm)]])
MM_pm = 1 / sqrt(2) * Matrix([[exp(- ttheta_pm), exp(ttheta_pm)], [exp(- ttheta_pm), - exp(ttheta_pm)]])
M_conj_pm = 1 / sqrt(2) * Matrix([[exp(theta_pm), exp(- theta_pm)], [exp(theta_pm), - exp(- theta_pm)]])
MM_conj_pm = 1 / sqrt(2) * Matrix([[exp(ttheta_pm), exp(- ttheta_pm)], [exp(ttheta_pm), - exp(- ttheta_pm)]])

tested_transformed_L_GP = evaluateExpr(MM * L_GP * Transpose(MM_conj) * ff)
tested_transformed_P_GP = evaluateExpr((processExpr(D(tt) * MM) * Transpose(M_conj) + MM * P_GP * Transpose(MM_conj)) * ff)

substituted_tested_transformed_L_GP = subs(tested_transformed_L_GP, [(q, a * exp(I * phi)), (q_conj, a * exp(- I * phi)), (deriv(phi), v)])
substituted_tested_transformed_P_GP = subs(tested_transformed_P_GP, [
    (phi, log(q / a) / I), 
    (hGP_a_LHS, hGP_a_RHS), 
    (simplify(GP_q_LHS / I), GP_q_RHS / I), 
    (simplify(GP_q_conj_LHS / I), GP_q_conj_RHS / I),
    (q, a * exp(I * phi)), 
    (q_conj, a * exp(- I * phi)),
    (deriv(phi), v)
])

tested_L_hGP_av = evaluateExpr(L_hGP_av * ff)
tested_P_hGP_av = evaluateExpr(P_hGP_av * ff)

display(Equ(Symbol('\\text{Error}_{\\text{transformed GP vs hGP Lax pair}}^L'), simplify(substituted_tested_transformed_L_GP - tested_L_hGP_av)))
display(Equ(Symbol('\\text{Error}_{\\text{transformed GP vs hGP Lax pair}}^P'), simplify(substituted_tested_transformed_P_GP - tested_P_hGP_av)))


Eq(\text{Error}_{\text{KdV vs Lax pair}}, 0)

Eq(\text{Error}_{\text{NLS vs Lax pair}}, Matrix([
[0, 0],
[0, 0]]))

Eq(\text{Error}_{\text{GP vs Lax pair}}, Matrix([
[0, 0],
[0, 0]]))

Eq(\text{Error}_{\text{hGP vs Lax pair}}^{(a, v)}, Matrix([
[0, 0],
[0, 0]]))

Eq(\text{Error}_{\text{hGP vs Lax pair}}^{(w_-, w_+)}, Matrix([
[0, 0],
[0, 0]]))

Eq(\text{Error}_{\text{rGP vs Lax pair}}^{(W_-, W_+)}, Matrix([
[0, 0],
[0, 0]]))

Eq(\text{Error}_{\text{transformed GP vs hGP Lax pair}}^L, Matrix([
[0, 0],
[0, 0]]))

Eq(\text{Error}_{\text{transformed GP vs hGP Lax pair}}^P, Matrix([
[0, 0],
[0, 0]]))

# Jost solutions - dark soliton case

In [ ]:
def mytanh(x):
    return (exp(x) - exp(-x)) / (exp(x) + exp(-x))
def mysech(x):
    return 2 / (exp(x) + exp(-x))

a, b, a_1, b_1, q_plus, q_minus = symbols('a b a_1 b_1 q_+ q_-', positive=True)
z = Symbol('z')
zeta_1 = Symbol('zeta_1')
zeta_1_conj = Symbol('\\overline{\\zeta_1}')
zeta = Symbol('zeta')
zeta_sub = lam + z_GP
zeta_1_sub = a_1 + I * b_1
zeta_1_conj_sub = a_1 - I * b_1
b_1_sub = sqrt(1 - a_1**2)
theta_1 = b_1 * (xx - 2 * a_1 * tt)
soliton = q_plus * zeta_1_conj * (a_1 + I * b_1 * mytanh(theta_1))
soliton_conj = 1 / q_plus * zeta_1 * (a_1 - I * b_1 * mytanh(theta_1))
soliton_sub = soliton.subs(zeta_1, zeta_1_sub).subs(zeta_1_conj, zeta_1_conj_sub)
soliton_conj_sub = soliton_conj.subs(zeta_1, zeta_1_sub).subs(zeta_1_conj, zeta_1_conj_sub)

display(soliton)
eqn = evaluateExpr((I * D(tt) - D(xx, xx)) * soliton_sub + 2 * (soliton_sub * soliton_conj_sub - 1) * soliton_sub).subs(b_1, - b_1_sub)
eqn = simplify(eqn)
display(eqn)


In [ ]:
ssoliton = simplify(soliton.subs(tt, 0))
ssoliton_conj = simplify(soliton_conj.subs(tt, 0))
theta_1 = simplify(theta_1.subs(tt, 0))


In [ ]:

mjost_11 = q_minus * (zeta - zeta_1) / (zeta - zeta_1_conj)
mjost_11 += I * q_minus / (zeta - zeta_1_conj) * b_1 * mysech(theta_1) * exp(- theta_1)

mjost_12 = I / zeta * (zeta - zeta_1) / (zeta - zeta_1_conj) * zeta_1_conj**2
mjost_12 += - zeta_1_conj / (zeta - zeta_1_conj) * b_1 * mysech(theta_1) * exp(- theta_1)

mjost_21 = - I / zeta
mjost_21 += - 1 / (zeta - zeta_1_conj) / zeta_1 * b_1 * mysech(theta_1) * exp(- theta_1)

mjost_22 = 1 / q_plus
mjost_22 += - I / q_plus / (zeta - zeta_1_conj) * b_1 * mysech(theta_1) * exp(- theta_1)

mjost_1 = Matrix([mjost_11, mjost_12])
mjost_2 = Matrix([mjost_21, mjost_22])

z_sub = (zeta - 1 / zeta) / 2
lam_sub = (zeta + 1 / zeta) / 2


In [ ]:
expr11 = evaluateExpr(I * Matrix([[D(xx) - I * z_sub, 0], [0, - D(xx) + I * z_sub]]) * mjost_1)
expr12 = evaluateExpr(I * Matrix([[0, - ssoliton], [ssoliton_conj, 0]]) * mjost_1)
expr13 = evaluateExpr(Matrix([[- lam_sub, 0], [0, - lam_sub]]) * mjost_1)

expr21 = evaluateExpr(I * Matrix([[D(xx) + I * z_sub, 0], [0, - D(xx) - I * z_sub]]) * mjost_2)
expr22 = evaluateExpr(I * Matrix([[0, - ssoliton], [ssoliton_conj, 0]]) * mjost_2)
expr23 = evaluateExpr(Matrix([[- lam_sub, 0], [0, - lam_sub]]) * mjost_2)

subs_list = [(q_plus, zeta_1**2 * q_minus), (zeta_1, zeta_1_sub), (zeta_1_conj, zeta_1_conj_sub), (b_1, - b_1_sub)]
exprall1 = simplify(subs(simplify(expr11 + expr12 + expr13), subs_list))
exprall2 = simplify(subs(simplify(expr21 + expr22 + expr23), subs_list))

display(exprall1)
display(exprall2)

In [ ]:
display(mjost.subs(xx, yy))

In [ ]:
mjost = evaluateExpr(Matrix([[mjost_11, mjost_21], [mjost_12, mjost_22]])).subs(zeta_1_conj, 1 / zeta_1)
mjost_inv = evaluateExpr(1 / det(E_GP) * Matrix([[- mjost_11, mjost_12], [mjost_21, - mjost_22]])).subs(zeta_1_conj, 1 / zeta_1)


In [ ]:
myprod = mjost.subs(x, y) * mjost_inv
display(myprod)

In [ ]:
myprod2 = subs(myprod, subs_list)

In [ ]:
display(myprod2)

In [ ]:
myprod3 = simplify(myprod2)

# Jost solutions - general case

In [8]:

Id = Matrix([[1, 0], [0, 1]])
Null = Matrix([[0, 0], [0, 0]])
sigma_2 = Matrix([[0, 1], [1, 0]])
sigma_3 = Matrix([[1, 0], [0, -1]])

def ssqrt(z):
    return - I * sqrt(- z)


rho_NLS = sigma_3
A_NLS = - I * lam * rho_NLS
AA_NLS = - I * lam * rho_NLS
U_NLS = Matrix([[0, q], [q_conj, 0]])
UU_NLS = Matrix([[0, qq], [qq_conj, 0]])
U_pm_NLS = Null
UU_pm_NLS = Null
z_NLS = lam
lam_NLS = z
E_NLS = Id
EE_NLS = Id
Lin_NLS = E_NLS * Matrix([[exp(- x * I * z_NLS), 0], [0, exp(x * I * z_NLS)]])
LLin_NLS = EE_NLS * Matrix([[exp(- xx * I * z_NLS), 0], [0, exp(xx * I * z_NLS)]])
L_pm_NLS = subs(L_NLS, [(qq, qq_pm), (qq_conj, 1 / qq_pm)])

obtained_Jost_NLS = evaluateExpr((- D(xx) * Id + AA_NLS + UU_NLS) * PPhi)
actual_Jost_NLS = evaluateExpr((L_NLS - lam * Id) * PPhi)

display(Equ(Symbol('\\text{Error}_{\\text{NLS Jost matches Lax}}'), simplify(actual_Jost_NLS + I * rho_NLS * obtained_Jost_NLS)))
display(Equ(Symbol('\\text{Error}_{\\text{NLS z matches eigenvalues}}'), simplify((A_NLS + U_pm_NLS).diagonalize()[1] + I * z_NLS * sigma_3)))
display(Equ(Symbol('\\text{Error}_{\\text{NLS linear solution vs semigroup}}'), simplify(Lin_NLS - exp(x * (A_NLS + U_pm_NLS)) * E_NLS)))
display(Equ(Symbol('\\text{Error}_{\\text{NLS linear solution is solution}}'), simplify(evaluateExpr((- D(xx) * Id + AA_NLS + UU_pm_NLS) * LLin_NLS))))


rho_GP = sigma_3
A_GP = - I * lam * rho_GP
AA_GP = - I * lam * rho_GP
U_GP = Matrix([[0, q], [q_conj, 0]])
UU_GP = Matrix([[0, qq], [qq_conj, 0]])
U_pm_GP = Matrix([[0, q_pm], [1 / q_pm, 0]])
UU_pm_GP = Matrix([[0, qq_pm], [1 / qq_pm, 0]])
z_GP = sqrt(lam**2 - 1)
# z_GP = I * sqrt(- lam**2 + 1)
#lam_GP = I * sqrt(- z**2 - 1)
lam_GP = sqrt(1 + z**2)
# E_GP = Matrix([[1, I * q_pm * (z_GP - lam)], [- I / q_pm * (z_GP - lam), 1]]) * sigma_3 * Matrix([[exp(theta_pm), 0], [0, exp(- theta_pm)]]) #Put these two matrices to hGP and the phase shift term in T shifts over
# EE_GP = Matrix([[1, I * qq_pm * (z_GP - lam)], [- I / qq_pm * (z_GP - lam), 1]]) * sigma_3 * Matrix([[exp(ttheta_pm), 0], [0, exp(- ttheta_pm)]])
E_GP = Matrix([[q_pm, I * (z_GP - lam)], [- I * (z_GP - lam), 1 / q_pm]])
EE_GP = Matrix([[qq_pm, I * (z_GP - lam)], [- I * (z_GP - lam), 1 / qq_pm]])
# E_GP = Matrix([[q_pm, I * (z_GP - lam)], [I * (lam - z_GP), 1 / q_pm]])
# EE_GP = Matrix([[qq_pm, - qq_pm], [- I * (zz_GP - llam), - I * (zz_GP + llam)]])
Lin_GP = E_GP * Matrix([[exp(- x * I * z_GP), 0], [0, exp(x * I * z_GP)]])
LLin_GP = EE_GP * Matrix([[exp(- xx * I * z_GP), 0], [0, exp(xx * I * z_GP)]])
L_pm_GP = subs(L_GP, [(qq, qq_pm), (qq_conj, 1 / qq_pm)])

obtained_Jost_GP = evaluateExpr((- D(xx) * Id + AA_GP + UU_GP) * PPhi)
actual_Jost_GP = evaluateExpr((L_GP - lam * Id) * PPhi)

display(Equ(Symbol('\\text{Error}_{\\text{GP Jost matches Lax}}'), simplify(actual_Jost_GP + I * rho_GP * obtained_Jost_GP)))
display(Equ(Symbol('\\text{Error}_{\\text{GP z matches eigenvalues}}'), tsimplify(sigma_2 * (A_GP + U_pm_GP).diagonalize()[1] * sigma_2 + I * z_GP * sigma_3)))
display(Equ(Symbol('\\text{Error}_{\\text{GP linear solution vs semigroup}}'), tsimplify(Lin_GP - exp(x * (A_GP + U_pm_GP)) * E_GP)))
display(Equ(Symbol('\\text{Error}_{\\text{GP linear solution is solution}}'), simplify(evaluateExpr((- D(xx) * Id + AA_GP + UU_pm_GP) * LLin_GP))))


rho_hGP = sigma_2
A_hGP = - I * lam * rho_hGP
AA_hGP = - I * lam * rho_hGP
U_hGP = Matrix([[0, - I * (w_plus + 1)], [- I * (w_minus - 1), 0]])
UU_hGP = Matrix([[0, - I * (ww_plus + 1)], [- I * (ww_minus - 1), 0]])
U_pm_hGP = Matrix([[0, - I], [I, 0]])
UU_pm_hGP = Matrix([[0, - I], [I, 0]])
z_hGP = sqrt(lam**2 - 1)
#lam_hGP = I * sqrt(- z**2 - 1)
lam_hGP = sqrt(1 + z**2)
# E_hGP = 1 / sqrt(2) * Matrix([[lam - z_hGP + 1, - lam + z_hGP - 1], [- lam + z_hGP + 1, - lam + z_hGP + 1]]) #* Matrix([[exp(- theta_pm), 0], [0, exp(theta_pm)]]) * sigma_3
# EE_hGP = 1 / sqrt(2) * Matrix([[lam - z_hGP + 1, - lam + z_hGP - 1], [- lam + z_hGP + 1, - lam + z_hGP + 1]]) #* Matrix([[exp(- ttheta_pm), 0], [0, exp(ttheta_pm)]]) * sigma_3
E_hGP = I / sqrt(2) * Matrix([[lam - z_hGP + 1, - lam + z_hGP - 1], [- lam + z_hGP + 1, - lam + z_hGP + 1]]) * Matrix([[exp(theta_pm), 0], [0, exp(- theta_pm)]])
EE_hGP = I / sqrt(2) * Matrix([[lam - z_hGP + 1, - lam + z_hGP - 1], [- lam + z_hGP + 1, - lam + z_hGP + 1]]) * Matrix([[exp(ttheta_pm), 0], [0, exp(- ttheta_pm)]])
Lin_hGP = E_hGP * Matrix([[exp(- x * I * z_hGP), 0], [0, exp(x * I * z_hGP)]])
LLin_hGP = EE_hGP * Matrix([[exp(- xx * I * z_hGP), 0], [0, exp(xx * I * z_hGP)]])
L_pm_hGP = subs(L_hGP_w, [(ww_minus, 0), (ww_plus, 0)])

obtained_Jost_hGP = evaluateExpr((- D(xx) * Id + AA_hGP + UU_hGP) * PPhi)
actual_Jost_hGP = evaluateExpr((L_hGP_w - lam * Id) * PPhi)

display(Equ(Symbol('\\text{Error}_{\\text{hGP Jost matches Lax}}'), simplify(actual_Jost_hGP + I * rho_hGP * obtained_Jost_hGP)))
display(Equ(Symbol('\\text{Error}_{\\text{hGP z matches eigenvalues}}'), tsimplify(sigma_2 * (A_hGP + U_pm_hGP).diagonalize()[1] * sigma_2 + I * z_hGP * sigma_3)))
display(Equ(Symbol('\\text{Error}_{\\text{hGP linear solution vs semigroup}}'), tsimplify(Lin_hGP - exp(x * (A_hGP + U_pm_hGP)) * E_hGP)))
display(Equ(Symbol('\\text{Error}_{\\text{hGP linear solution is solution}}'), simplify(evaluateExpr((- D(xx) * Id + AA_hGP + UU_pm_hGP) * LLin_hGP))))


rho_rGP = sigma_2
A_rGP = - I * lam / sqrt(2) / epsilon * rho_rGP
AA_rGP = - I * lam / sqrt(2) / epsilon * rho_rGP
U_rGP = Matrix([[0, - I * epsilon / sqrt(2) * (W_plus + 1 / epsilon**2)], [- I * epsilon / sqrt(2) * (W_minus - 1 / epsilon**2), 0]])
UU_rGP = Matrix([[0, - I * epsilon / sqrt(2) * (WW_plus + 1 / epsilon**2)], [- I * epsilon / sqrt(2) * (WW_minus - 1 / epsilon**2), 0]])
U_pm_rGP = Matrix([[0, - I / epsilon / sqrt(2)], [I / epsilon / sqrt(2), 0]])
UU_pm_rGP = Matrix([[0, - I / epsilon / sqrt(2)], [I / epsilon / sqrt(2), 0]])
z_rGP = sqrt(lam**2 - 1) / sqrt(2) / epsilon
#lam_rGP = I * sqrt(- 2 * epsilon**2 * z**2 - 1)
lam_rGP = sqrt(1 + 2 * epsilon**2 * z**2)
# E_rGP = 1 / sqrt(2) * Matrix([
#     [lam - sqrt(2) * epsilon * z_rGP + 1, - lam + sqrt(2) * epsilon * z_rGP - 1], 
#     [- lam + sqrt(2) * epsilon * z_rGP + 1, - lam + sqrt(2) * epsilon * z_rGP + 1]
# ]) #* Matrix([[exp(- theta_pm), 0], [0, exp(theta_pm)]]) * sigma_3
# EE_rGP = 1 / sqrt(2) * Matrix([
#     [lam - sqrt(2) * epsilon * z_rGP + 1, - lam + sqrt(2) * epsilon * z_rGP - 1],
#     [- lam + sqrt(2) * epsilon * z_rGP + 1, - lam + sqrt(2) * epsilon * z_rGP + 1]
# ]) #* Matrix([[exp(- ttheta_pm), 0], [0, exp(ttheta_pm)]]) * sigma_3
E_rGP = I / sqrt(2) * Matrix([
    [lam - sqrt(2) * epsilon * z_rGP + 1, - lam + sqrt(2) * epsilon * z_rGP - 1], 
    [- lam + sqrt(2) * epsilon * z_rGP + 1, - lam + sqrt(2) * epsilon * z_rGP + 1]
]) * Matrix([[exp(theta_pm), 0], [0, exp(- theta_pm)]])
EE_rGP = I / sqrt(2) * Matrix([
    [lam - sqrt(2) * epsilon * z_rGP + 1, - lam + sqrt(2) * epsilon * z_rGP - 1],
    [- lam + sqrt(2) * epsilon * z_rGP + 1, - lam + sqrt(2) * epsilon * z_rGP + 1]
]) * Matrix([[exp(ttheta_pm), 0], [0, exp(- ttheta_pm)]])
Lin_rGP = E_rGP * Matrix([[exp(- x * I * z_rGP), 0], [0, exp(x * I * z_rGP)]])
LLin_rGP = EE_rGP * Matrix([[exp(- xx * I * z_rGP), 0], [0, exp(xx * I * z_rGP)]])
L_pm_rGP = subs(L_hGP_w, [(WW_minus, 0), (WW_plus, 0)])

obtained_Jost_rGP = evaluateExpr((- D(xx) * Id + AA_rGP + UU_rGP) * PPhi)
actual_Jost_rGP = evaluateExpr((L_rGP_W - lam / epsilon**2 * Id) * PPhi)

display(Equ(Symbol('\\text{Error}_{\\text{rGP Jost matches Lax}}'), simplify(actual_Jost_rGP + sqrt(2) / epsilon * I * rho_rGP * obtained_Jost_rGP)))
display(Equ(Symbol('\\text{Error}_{\\text{rGP z matches eigenvalues}}'), tsimplify(sigma_2 * (A_rGP + U_pm_rGP).diagonalize()[1] * sigma_2 + I * z_rGP * sigma_3)))
display(Equ(Symbol('\\text{Error}_{\\text{rGP linear solution vs semigroup}}'), tsimplify(Lin_rGP - exp(x * (A_rGP + U_pm_rGP)) * E_rGP)))
display(Equ(Symbol('\\text{Error}_{\\text{rGP linear solution is solution}}'), simplify(evaluateExpr((- D(xx) * Id + AA_rGP + UU_pm_rGP) * LLin_rGP))))


Jost_GP_to_hGP = subs(evaluateExpr(I * rho_GP * (- D(xx) * Id + AA_GP + UU_GP) * PPhi), [
    (q, (w_plus - w_minus + 2) / 2 * exp(I * phi)),
    (q_conj, (w_plus - w_minus + 2) / 2 * exp(- I * phi)),
    (deriv(phi), w_minus + w_plus)
])
Jost_hGP = subs(evaluateExpr(Transpose(MM_conj) * I * rho_hGP * (- D(xx) * Id + AA_hGP + UU_hGP) * MM * PPhi), [
    (deriv(phi), w_minus + w_plus)
])

display(Equ(Symbol('\\text{Error}_{\\text{hGP Jost solution from GP Jost solution}}'), simplify(Jost_GP_to_hGP - Jost_hGP)))

display(Equ(Symbol('\\text{Error}_{\\text{hGP linear solution from GP linear solution}}'), 
            tsimplify(subs(M_pm * Lin_GP, [(q_pm, exp(I * phi_pm))]) - Lin_hGP)))


Jost_GP_to_rGP = subs(evaluateExpr((- D(xx) * Id + AA_GP + UU_GP) * PPhi), [
    (q, (w_plus - w_minus + 2) / 2 * exp(I * phi)),
    (q_conj, (w_plus - w_minus + 2) / 2 * exp(- I * phi)),
    (deriv(phi), w_minus + w_plus),
    (w_minus, epsilon**2 * W_minus), 
    (w_plus, epsilon**2 * W_plus)
], scale={
    x: epsilon * sqrt(2)
})
Jost_rGP = subs(subs(evaluateExpr(rho_GP * Transpose(MM_conj) * sqrt(2) * epsilon * rho_rGP * (- D(xx) * Id + AA_rGP + UU_rGP) * MM * PPhi), [
    (phi, phi), (Phi_11, Phi_11), (Phi_12, Phi_12), (Phi_21, Phi_21), (Phi_22, Phi_22),
], scale={
    x: epsilon**(-1) / sqrt(2)
}), [(deriv(phi), epsilon**2 * (W_minus + W_plus))])

display(Equ(Symbol('\\text{Error}_{\\text{rGP Jost solution from GP Jost solution}}'), simplify(Jost_GP_to_rGP - Jost_rGP)))

display(Equ(Symbol('\\text{Error}_{\\text{rGP linear solution from GP linear solution}}'), 
            simplify(subs(M_pm * Lin_GP, [(q_pm, exp(I * phi_pm)), (x, x / sqrt(2) / epsilon)]) - Lin_rGP)))





Eq(\text{Error}_{\text{NLS Jost matches Lax}}, Matrix([
[0, 0],
[0, 0]]))

Eq(\text{Error}_{\text{NLS z matches eigenvalues}}, Matrix([
[0, 0],
[0, 0]]))

Eq(\text{Error}_{\text{NLS linear solution vs semigroup}}, Matrix([
[0, 0],
[0, 0]]))

Eq(\text{Error}_{\text{NLS linear solution is solution}}, Matrix([
[0, 0],
[0, 0]]))

Eq(\text{Error}_{\text{GP Jost matches Lax}}, Matrix([
[0, 0],
[0, 0]]))

...

Eq(\text{Error}_{\text{rGP linear solution is solution}}, Matrix([
[0, 0],
[0, 0]]))

Eq(\text{Error}_{\text{hGP Jost solution from GP Jost solution}}, Matrix([
[0, 0],
[0, 0]]))

Eq(\text{Error}_{\text{hGP linear solution from GP linear solution}}, Matrix([
[0, 0],
[0, 0]]))

Eq(\text{Error}_{\text{rGP Jost solution from GP Jost solution}}, Matrix([
[0, 0],
[0, 0]]))

Eq(\text{Error}_{\text{rGP linear solution from GP linear solution}}, Matrix([
[0, 0],
[0, 0]]))

# Transmission Coefficient

In [9]:
E_minus_NLS = subs(E_NLS, [(q_pm, exp(I * phi_minus)), (phi_pm, phi_minus)])
E_plus_NLS = subs(E_NLS, [(q_pm, exp(I * phi_plus)), (phi_pm, phi_plus)])
E_minus_GP = subs(E_GP, [(q_pm, exp(I * phi_minus)), (phi_pm, phi_minus)])
E_plus_GP = subs(E_GP, [(q_pm, exp(I * phi_plus)), (phi_pm, phi_plus)])
E_minus_hGP = subs(E_hGP, [(phi_pm, phi_minus)])
E_plus_hGP = subs(E_hGP, [(phi_pm, phi_plus)])
E_minus_rGP = subs(E_rGP, [(phi_pm, phi_minus)])
E_plus_rGP = subs(E_rGP, [(phi_pm, phi_plus)])

T_boundary_NLS = simplify(log(E_minus_NLS[0,0]) - log(E_plus_NLS[0,0])).expand(force=True)
T_integrand_NLS = deriv(Phi[0,0]) / Phi[0,0] + I * z_NLS
T_boundary_GP = simplify(log(E_minus_GP[0,0]) - log(E_plus_GP[0,0])).expand(force=True)
T_integrand_GP = deriv(Phi[0,0]) / Phi[0,0] + I * z_GP
T_boundary_hGP = simplify(log(E_minus_hGP[0,0]) - log(E_plus_hGP[0,0])).expand(force=True)
T_integrand_hGP = deriv((M * Phi)[0,0]) / (M * Phi)[0,0] + I * z_hGP
T_boundary_rGP = simplify(log(E_minus_rGP[0,0]) - log(E_plus_rGP[0,0])).expand(force=True)
T_integrand_rGP = subs(deriv((M * Phi)[0,0]) / (M * Phi)[0,0], [
    (Phi_11, Phi_11), 
    (Phi_12, Phi_12), 
    (phi, phi)
], scale={x: 1 / sqrt(2) / epsilon}) + I * z_rGP

print("boundary terms: ")
display(T_boundary_NLS, T_boundary_GP, T_boundary_hGP, T_boundary_rGP)

M_minus = subs(M, [(phi, phi_minus)])
M_plus = subs(M, [(phi, phi_plus)])

sig_LHS_NLS = subs(deriv(sig), [(lam, lam_NLS)])
sig_RHS_NLS = subs(- sig**2 + (2 * I * z_NLS) * sig + z_NLS**2 + (deriv(U_NLS) + (A_NLS + U_NLS)**2)[0,0] + (deriv(U_NLS) + (A_NLS + U_NLS)**2)[0,1] * r, [(lam, lam_NLS)])
r_LHS_NLS = subs(deriv(r), [(lam, lam_NLS)])
r_RHS_NLS = subs(- (A_NLS + U_NLS)[0,1] * r**2 + ((A_NLS + U_NLS)[1,1] - (A_NLS + U_NLS)[0,0]) * r + (A_NLS + U_NLS)[1,0], [(lam, lam_NLS)])

substituted_sig_RHS_NLS = subs(sig_RHS_NLS, [
    (sig, deriv(Phi_11) / Phi_11 + I * z_NLS), 
    (r, Phi_12 / Phi_11),
    (deriv(Phi_11), ((A_NLS + U_NLS) * Phi)[0,0]),
    (deriv(Phi_12), ((A_NLS + U_NLS) * Phi)[1,0]),
    (deriv(Phi_11), ((A_NLS + U_NLS) * Phi)[0,0]),
    (deriv(Phi_12), ((A_NLS + U_NLS) * Phi)[1,0]),
    (lam, lam_NLS)
])
substituted_sig_LHS_NLS = subs(sig_LHS_NLS, [
    (sig, deriv(Phi_11) / Phi_11 + I * z_NLS), 
    (r, Phi_12 / Phi_11),
    (deriv(Phi_11), ((A_NLS + U_NLS) * Phi)[0,0]),
    (deriv(Phi_12), ((A_NLS + U_NLS) * Phi)[1,0]),
    (deriv(Phi_11), ((A_NLS + U_NLS) * Phi)[0,0]),
    (deriv(Phi_12), ((A_NLS + U_NLS) * Phi)[1,0]),
    (lam, lam_NLS)
])
substituted_r_RHS_NLS = subs(r_RHS_NLS, [
    (sig, deriv(Phi_11) / Phi_11 + I * z_NLS), 
    (r, Phi_12 / Phi_11),
    (deriv(Phi_11), ((A_NLS + U_NLS) * Phi)[0,0]),
    (deriv(Phi_12), ((A_NLS + U_NLS) * Phi)[1,0]),
    (lam, lam_NLS)
])
substituted_r_LHS_NLS = subs(r_LHS_NLS, [
    (sig, deriv(Phi_11) / Phi_11 + I * z_NLS), 
    (r, Phi_12 / Phi_11),
    (deriv(Phi_11), ((A_NLS + U_NLS) * Phi)[0,0]),
    (deriv(Phi_12), ((A_NLS + U_NLS) * Phi)[1,0]),
    (lam, lam_NLS)
])

display(Equ(Symbol('\\text{Error}_{\\text{NLS formula for $\\sigma$}}'), simplify(substituted_sig_LHS_NLS - substituted_sig_RHS_NLS)))
display(Equ(Symbol('\\text{Error}_{\\text{NLS formula for $r$}}'), simplify(substituted_r_LHS_NLS - substituted_r_RHS_NLS)))


sig_LHS_GP = subs(deriv(sig), [(lam, lam_GP)])
sig_RHS_GP = subs(- sig**2 + (2 * I * z_GP) * sig + z_GP**2 + (deriv(U_GP) + (A_GP + U_GP)**2)[0,0] + (deriv(U_GP) + (A_GP + U_GP)**2)[0,1] * r, [(lam, lam_GP)])
r_LHS_GP = subs(deriv(r), [(lam, lam_GP)])
r_RHS_GP = subs(- (A_GP + U_GP)[0,1] * r**2 + ((A_GP + U_GP)[1,1] - (A_GP + U_GP)[0,0]) * r + (A_GP + U_GP)[1,0], [(lam, lam_GP)])

substituted_sig_RHS_GP = subs(sig_RHS_GP, [
    (sig, deriv(Phi_11) / Phi_11 + I * z_GP), 
    (r, Phi_12 / Phi_11),
    (deriv(Phi_11), ((A_GP + U_GP) * Phi)[0,0]),
    (deriv(Phi_12), ((A_GP + U_GP) * Phi)[1,0]),
    (deriv(Phi_11), ((A_GP + U_GP) * Phi)[0,0]),
    (deriv(Phi_12), ((A_GP + U_GP) * Phi)[1,0]),
    (lam, lam_GP)
])
substituted_sig_LHS_GP = subs(sig_LHS_GP, [
    (sig, deriv(Phi_11) / Phi_11 + I * z_GP), 
    (r, Phi_12 / Phi_11),
    (deriv(Phi_11), ((A_GP + U_GP) * Phi)[0,0]),
    (deriv(Phi_12), ((A_GP + U_GP) * Phi)[1,0]),
    (deriv(Phi_11), ((A_GP + U_GP) * Phi)[0,0]),
    (deriv(Phi_12), ((A_GP + U_GP) * Phi)[1,0]),
    (lam, lam_GP)
])
substituted_r_RHS_GP = subs(r_RHS_GP, [
    (sig, deriv(Phi_11) / Phi_11 + I * z_GP), 
    (r, Phi_12 / Phi_11),
    (deriv(Phi_11), ((A_GP + U_GP) * Phi)[0,0]),
    (deriv(Phi_12), ((A_GP + U_GP) * Phi)[1,0]),
    (lam, lam_GP)
])
substituted_r_LHS_GP = subs(r_LHS_GP, [
    (sig, deriv(Phi_11) / Phi_11 + I * z_GP), 
    (r, Phi_12 / Phi_11),
    (deriv(Phi_11), ((A_GP + U_GP) * Phi)[0,0]),
    (deriv(Phi_12), ((A_GP + U_GP) * Phi)[1,0]),
    (lam, lam_GP)
])


display(Equ(Symbol('\\text{Error}_{\\text{GP formula for $\\sigma$}}'), simplify(substituted_sig_LHS_GP - substituted_sig_RHS_GP)))
display(Equ(Symbol('\\text{Error}_{\\text{GP formula for $r$}}'), simplify(substituted_r_LHS_GP - substituted_r_RHS_GP)))


sig_LHS_hGP = subs(deriv(sig), [(lam, lam_hGP)])
sig_RHS_hGP = subs(- sig**2 + (2 * I * z_hGP) * sig + z_hGP**2 + (deriv(U_hGP) + (A_hGP + U_hGP)**2)[0,0] + (deriv(U_hGP) + (A_hGP + U_hGP)**2)[0,1] * r, [(lam, lam_hGP)])
r_LHS_hGP = subs(deriv(r), [(lam, lam_hGP)])
r_RHS_hGP = subs(- (A_hGP + U_hGP)[0,1] * r**2 + ((A_hGP + U_hGP)[1,1] - (A_hGP + U_hGP)[0,0]) * r + (A_hGP + U_hGP)[1,0], [(lam, lam_hGP)])

substituted_sig_RHS_hGP = subs(sig_RHS_hGP, [
    (sig, deriv(Phi_11) / Phi_11 + I * z_hGP), 
    (r, Phi_12 / Phi_11),
    (deriv(Phi_11), ((A_hGP + U_hGP) * Phi)[0,0]),
    (deriv(Phi_12), ((A_hGP + U_hGP) * Phi)[1,0]),
    (lam, lam_hGP)
])
substituted_sig_LHS_hGP = subs(sig_LHS_hGP, [
    (sig, deriv(Phi_11) / Phi_11 + I * z_hGP), 
    (r, Phi_12 / Phi_11),
    (deriv(Phi_11), ((A_hGP + U_hGP) * Phi)[0,0]),
    (deriv(Phi_12), ((A_hGP + U_hGP) * Phi)[1,0]),
    (lam, lam_hGP)
])
substituted_r_RHS_hGP = subs(r_RHS_hGP, [
    (sig, deriv(Phi_11) / Phi_11 + I * z_hGP), 
    (r, Phi_12 / Phi_11),
    (deriv(Phi_11), ((A_hGP + U_hGP) * Phi)[0,0]),
    (deriv(Phi_12), ((A_hGP + U_hGP) * Phi)[1,0]),
    (lam, lam_hGP)
])
substituted_r_LHS_hGP = subs(r_LHS_hGP, [
    (sig, deriv(Phi_11) / Phi_11 + I * z_hGP), 
    (r, Phi_12 / Phi_11),
    (deriv(Phi_11), ((A_hGP + U_hGP) * Phi)[0,0]),
    (deriv(Phi_12), ((A_hGP + U_hGP) * Phi)[1,0]),
    (lam, lam_hGP)
])

display(Equ(Symbol('\\text{Error}_{\\text{hGP formula for $\\sigma$}}'), simplify(substituted_sig_LHS_hGP - substituted_sig_RHS_hGP)))
display(Equ(Symbol('\\text{Error}_{\\text{hGP formula for $r$}}'), simplify(substituted_r_LHS_hGP - substituted_r_RHS_hGP)))


sig_LHS_rGP = subs(deriv(sig), [(lam, lam_rGP)])
sig_RHS_rGP = subs(- sig**2 + (2 * I * z_rGP) * sig + z_rGP**2 + (deriv(U_rGP) + (A_rGP + U_rGP)**2)[0,0] + (deriv(U_rGP) + (A_rGP + U_rGP)**2)[0,1] * r, [(lam, lam_rGP)])
r_LHS_rGP = subs(deriv(r), [(lam, lam_rGP)])
r_RHS_rGP = subs(- (A_rGP + U_rGP)[0,1] * r**2 + ((A_rGP + U_rGP)[1,1] - (A_rGP + U_rGP)[0,0]) * r + (A_rGP + U_rGP)[1,0], [(lam, lam_rGP)])
sigt_LHS_rGP = subs(- deriv(sigt), [(lam, lam_rGP)])
sigt_RHS_rGP = subs(- sigt**2 + (2 * I * z_rGP) * sigt + z_rGP**2 + (deriv(U_rGP) + (A_rGP + U_rGP)**2)[1,1] + (deriv(U_rGP) + (A_rGP + U_rGP)**2)[1,0] * rt, [(lam, lam_rGP)])
rt_LHS_rGP = subs(deriv(rt), [(lam, lam_rGP)])
rt_RHS_rGP = subs(- (A_rGP + U_rGP)[1,0] * rt**2 + ((A_rGP + U_rGP)[0,0] - (A_rGP + U_rGP)[1,1]) * rt + (A_rGP + U_rGP)[0,1], [(lam, lam_rGP)])

subs_list = [
    (sig, deriv(Phi_11) / Phi_11 + I * z_rGP), 
    (r, Phi_12 / Phi_11),
    (deriv(Phi_11), ((A_rGP + U_rGP) * Phi)[0,0]),
    (deriv(Phi_12), ((A_rGP + U_rGP) * Phi)[1,0]),
    (lam, lam_rGP)
]
substituted_sig_RHS_rGP = subs(sig_RHS_rGP, subs_list)
substituted_sig_LHS_rGP = subs(sig_LHS_rGP, subs_list)
substituted_r_RHS_rGP = subs(r_RHS_rGP, subs_list)
substituted_r_LHS_rGP = subs(r_LHS_rGP, subs_list)

subs_list = [
    (sigt, - deriv(Phi_22) / Phi_22 + I * z_rGP), 
    (rt, Phi_21 / Phi_22),
    (deriv(Phi_22), ((A_rGP + U_rGP) * Phi)[1,1]),
    (deriv(Phi_21), ((A_rGP + U_rGP) * Phi)[0,1]),
    (lam, lam_rGP)
]
substituted_sigt_RHS_rGP = subs(sigt_RHS_rGP, subs_list)
substituted_sigt_LHS_rGP = subs(sigt_LHS_rGP, subs_list)
substituted_rt_RHS_rGP = subs(rt_RHS_rGP, subs_list)
substituted_rt_LHS_rGP = subs(rt_LHS_rGP, subs_list)

display(Equ(Symbol('\\text{Error}_{\\text{rGP formula for $\\sigma$}}'), simplify(substituted_sig_LHS_rGP - substituted_sig_RHS_rGP)))
display(Equ(Symbol('\\text{Error}_{\\text{rGP formula for $r$}}'), simplify(substituted_r_LHS_rGP - substituted_r_RHS_rGP)))
display(Equ(Symbol('\\text{Error}_{\\text{rGP formula for $\\tilde{\\sigma}$}}'), simplify(substituted_sigt_LHS_rGP - substituted_sigt_RHS_rGP)))
display(Equ(Symbol('\\text{Error}_{\\text{rGP formula for $\\tilde{r}$}}'), simplify(substituted_rt_LHS_rGP - substituted_rt_RHS_rGP)))


sig_eqn_NLS = tsimplify((- sig_LHS_NLS + sig_RHS_NLS).expand())
r_eqn_NLS = tsimplify((- r_LHS_NLS + r_RHS_NLS).expand())
sig_eqn_GP = tsimplify((- sig_LHS_GP+ sig_RHS_GP).expand())
r_eqn_GP = tsimplify((- r_LHS_GP + r_RHS_GP).expand())
sig_eqn_hGP= tsimplify((- sig_LHS_hGP + sig_RHS_hGP).expand())
r_eqn_hGP =tsimplify((- r_LHS_hGP + r_RHS_hGP).expand())
sig_eqn_rGP = tsimplify((- sig_LHS_rGP + sig_RHS_rGP).expand())
r_eqn_rGP = tsimplify((- r_LHS_rGP + r_RHS_rGP).expand())
sigt_eqn_rGP = tsimplify((- sigt_LHS_rGP + sigt_RHS_rGP).expand())
rt_eqn_rGP = tsimplify((- rt_LHS_rGP + rt_RHS_rGP).expand())

sig_eqn_lam_NLS = tsimplify((- sig_LHS_NLS + sig_RHS_NLS).subs(z, z_NLS).expand())
r_eqn_lam_NLS = tsimplify((- r_LHS_NLS + r_RHS_NLS).subs(z, z_NLS).expand())
sig_eqn_lam_GP = tsimplify((- sig_LHS_GP+ sig_RHS_GP).subs(z, z_GP).expand())
r_eqn_lam_GP = tsimplify((- r_LHS_GP + r_RHS_GP).subs(z, z_GP).expand())
sig_eqn_lam_hGP= tsimplify((- sig_LHS_hGP + sig_RHS_hGP).subs(z, z_hGP).expand())
r_eqn_lam_hGP =tsimplify((- r_LHS_hGP + r_RHS_hGP).subs(z, z_hGP).expand())
sig_eqn_lam_rGP = tsimplify((- sig_LHS_rGP + sig_RHS_rGP).subs(z, z_rGP).expand())
r_eqn_lam_rGP = tsimplify((- r_LHS_rGP + r_RHS_rGP).subs(z, z_rGP).expand())
sigt_eqn_lam_rGP = tsimplify((- sigt_LHS_rGP + sigt_RHS_rGP).subs(z, z_rGP).expand())
rt_eqn_lam_rGP = tsimplify((- rt_LHS_rGP + rt_RHS_rGP).subs(z, z_rGP).expand())


boundary terms: 


0

-I*\phi_+ + I*\phi_-

-I*\phi_+/2 + I*\phi_-/2

-I*\phi_+/2 + I*\phi_-/2

...

Eq(\text{Error}_{\text{hGP formula for $r$}}, 0)

Eq(\text{Error}_{\text{rGP formula for $\sigma$}}, 0)

Eq(\text{Error}_{\text{rGP formula for $r$}}, 0)

Eq(\text{Error}_{\text{rGP formula for $\tilde{\sigma}$}}, 0)

Eq(\text{Error}_{\text{rGP formula for $\tilde{r}$}}, 0)

# Approximation formal

In [10]:
mmu, mu, nnu, nu, rrho, rho, ttheta, theta, oomega, omega, ttau, tau, aalpha, alpha, bbeta, beta = D.create_diff_symbols(
    '\\mu', '\\nu', '\\rho', '\\theta', '\\omega', '\\tau', '\\alpha', '\\beta'
)
mmut, mut, nnut, nut, rrhot, rhot, tthetat, thetat, oomegat, omegat, ttaut, taut, aalphat, alphat, bbetat, betat = D.create_diff_symbols(
    '\\tilde{\\mu}', '\\tilde{\\nu}', '\\tilde{\\rho}', '\\tilde{\\theta}', '\\tilde{\\omega}', '\\tilde{\\tau}', '\\tilde{\\alpha}', '\\tilde{\\beta}'
)

# sig_eqn_lam_rGP = tsimplify((sig_LHS_rGP - sig_RHS_rGP).subs(z, z_rGP))
# sigt_eqn_lam_rGP = tsimplify((sigt_LHS_rGP - sigt_RHS_rGP).subs(z, z_rGP))
# r_eqn_lam_rGP = tsimplify((r_LHS_rGP - r_RHS_rGP).subs(z, z_rGP))
# rt_eqn_lam_rGP = tsimplify((rt_LHS_rGP - rt_RHS_rGP).subs(z, z_rGP))

mu_sub = re(sig) + 1 / (I * lam) * im(sig)
mut_sub = re(sigt) + 1 / (I * lam) * im(sigt)
nu_sub = re(sig) - 1 / (I * lam) * im(sig)
nut_sub = re(sigt) - 1 / (I * lam) * im(sigt)
rho_sub = re(sig)
rhot_sub = re(sigt)
theta_sub = 1 / (I * lam) * im(sig)
thetat_sub = 1 / (I * lam) * im(sigt)

alpha_sub = re(I * epsilon * r) + 1 / (I * lam) * im(I * epsilon * r)
alphat_sub = re(I * epsilon * rt) + 1 / (I * lam) * im(I * epsilon * rt)
beta_sub = re(I * epsilon * r) - 1 / (I * lam) * im(I * epsilon * r)
betat_sub = re(I * epsilon * rt) - 1 / (I * lam) * im(I * epsilon * rt)
omega_sub = re(I * epsilon * r)
omegat_sub = re(I * epsilon * rt)
tau_sub = 1 / (I * lam) * im(I * epsilon * r)
taut_sub = 1 / (I * lam) * im(I * epsilon * rt)

mu_eqn = tsimplify(mu_sub.subs(sig, - sig_eqn_lam_rGP))
mut_eqn = tsimplify(mut_sub.subs(sigt, - sigt_eqn_lam_rGP))
nu_eqn = tsimplify(nu_sub.subs(sig, - sig_eqn_lam_rGP))
nut_eqn = tsimplify(nut_sub.subs(sigt, - sigt_eqn_lam_rGP))
rho_eqn = tsimplify(rho_sub.subs(sig, - sig_eqn_lam_rGP))
rhot_eqn = tsimplify(rhot_sub.subs(sigt, - sigt_eqn_lam_rGP))
theta_eqn = tsimplify(theta_sub.subs(sig, - sig_eqn_lam_rGP))
thetat_eqn = tsimplify(thetat_sub.subs(sigt, - sigt_eqn_lam_rGP))
alpha_eqn = sqrt(2) * tsimplify(alpha_sub.subs(r, - r_eqn_lam_rGP))
alphat_eqn = sqrt(2) * tsimplify(alphat_sub.subs(rt, - rt_eqn_lam_rGP))
beta_eqn = sqrt(2) * tsimplify(beta_sub.subs(r, - r_eqn_lam_rGP))
betat_eqn = sqrt(2) * tsimplify(betat_sub.subs(rt, - rt_eqn_lam_rGP))
omega_eqn = sqrt(2) * tsimplify(omega_sub.subs(r, - r_eqn_lam_rGP))
omegat_eqn = sqrt(2) * tsimplify(omegat_sub.subs(rt, - rt_eqn_lam_rGP))
tau_eqn = sqrt(2) * tsimplify(tau_sub.subs(r, - r_eqn_lam_rGP))
taut_eqn = sqrt(2) * tsimplify(taut_sub.subs(rt, - rt_eqn_lam_rGP))

subs_list = [
    (z, z_rGP),
    (mu, mu_sub), (mut, mut_sub),
    (nu, nu_sub), (nut, nut_sub),
    (rho, rho_sub), (rhot, rhot_sub),
    (theta, theta_sub), (thetat, thetat_sub),
    (alpha, alpha_sub), (alphat, alphat_sub),
    (beta, beta_sub), (betat, betat_sub),
    (omega, omega_sub), (omegat, omegat_sub),
    (tau, tau_sub), (taut, taut_sub)
]
mu_eqn_expected = deriv(mu) - (2*I*z) * mu + mu**2 - epsilon**2 / 2 * (2*I*z)**2 * theta**2 - W_plus + epsilon**2 / 2 * W_minus * W_plus + deriv(W_plus) / sqrt(2) * alpha  
mut_eqn_expected = - deriv(mut) - (2*I*z) * mut + mut**2 - epsilon**2 / 2 * (2*I*z)**2 * thetat**2 - W_plus + epsilon**2 / 2 * W_minus * W_plus + deriv(W_minus) / sqrt(2) * alphat  
nu_eqn_expected = deriv(nu) - (2*I*z) * nu + nu**2 - epsilon**2 / 2 * (2*I*z)**2 * theta**2 + W_minus + epsilon**2 / 2 * W_minus * W_plus + deriv(W_plus) / sqrt(2) * beta
nut_eqn_expected = - deriv(nut) - (2*I*z) * nut + nut**2 - epsilon**2 / 2 * (2*I*z)**2 * thetat**2 + W_minus + epsilon**2 / 2 * W_minus * W_plus + deriv(W_minus) / sqrt(2) * betat
rho_eqn_expected = deriv(rho) - (2*I*z) * rho + rho**2 + theta**2 - epsilon**2 / 2 * (2*I*z)**2 * theta**2 + epsilon**2 / 2 * W_minus * W_plus + W_minus / 2 - W_plus / 2 + deriv(W_plus) / sqrt(2) * omega
rhot_eqn_expected = - deriv(rhot) - (2*I*z) * rhot + rhot**2 + thetat**2 - epsilon**2 / 2 * (2*I*z)**2 * thetat**2 + epsilon**2 / 2 * W_minus * W_plus + W_minus / 2 - W_plus / 2 + deriv(W_minus) / sqrt(2) * omegat
theta_eqn_expected = deriv(theta) - (2*I*z) * theta + 2 * rho * theta - W_minus / 2 - W_plus / 2 + deriv(W_plus) / sqrt(2) * tau
thetat_eqn_expected = - deriv(thetat) - (2*I*z) * thetat + 2 * rhot * thetat - W_minus / 2 - W_plus / 2 + deriv(W_minus) / sqrt(2) * taut
omega_eqn_expected = sqrt(2) * deriv(omega) + (2*I*z)**2 / 2 * tau**2 - W_plus * (omega**2 + tau**2 - epsilon**2 / 2 * (2*I*z)**2 * tau**2) - (2*I*z)**2 * omega * tau + 1 - epsilon**(-2) * beta**2 - epsilon**2 * W_minus
omegat_eqn_expected = sqrt(2) * deriv(omegat) - (2*I*z)**2 / 2 * taut**2 - W_minus * (omegat**2 + taut**2 - epsilon**2 / 2 * (2*I*z)**2 * taut**2) - (2*I*z)**2 * omegat * taut - 1 + epsilon**(-2) * alphat**2 - epsilon**2 * W_plus
tau_eqn_expected = sqrt(2) * deriv(tau) - (2*I*z)**2 / 2 * tau**2 - 2 * W_plus * omega * tau + 1 + epsilon**(-2) * beta**2
taut_eqn_expected = sqrt(2) * deriv(taut) - (2*I*z)**2 / 2 * taut**2 - 2 *  W_minus * omegat * taut + 1 + epsilon**(-2) * alphat**2
alpha_eqn_expected = sqrt(2) * deriv(alpha) - W_plus * alpha**2 + (2*I*z)**2 * (epsilon**2 / 2 * W_plus * tau**2 - omega * tau) + 2 - epsilon**2 * W_minus
alphat_eqn_expected = sqrt(2) * deriv(alphat) - (W_minus - 2 * epsilon**(-2)) * alphat**2 + (2*I*z)**2 * (epsilon**2 / 2 * W_minus * taut**2 - alphat * taut) - epsilon**2 * W_plus
beta_eqn_expected = sqrt(2) * deriv(beta) - (W_plus + 2 * epsilon**(-2)) * beta**2 + (2*I*z)**2 * (epsilon**2 / 2 * W_plus * tau**2 - beta * tau) - epsilon**2 * W_minus
betat_eqn_expected = sqrt(2) * deriv(betat) - W_minus * betat**2 + (2*I*z)**2 * (epsilon**2 / 2 * W_minus * taut**2 - omegat * taut) - 2 - epsilon**2 * W_plus

mu_eqn_expected_substituted = tsimplify(subs(mu_eqn_expected, subs_list))
mut_eqn_expected_substituted = tsimplify(subs(mut_eqn_expected, subs_list))
nu_eqn_expected_substituted = tsimplify(subs(nu_eqn_expected, subs_list))
nut_eqn_expected_substituted = tsimplify(subs(nut_eqn_expected, subs_list))
rho_eqn_expected_substituted = tsimplify(subs(rho_eqn_expected, subs_list))
rhot_eqn_expected_substituted = tsimplify(subs(rhot_eqn_expected, subs_list))
theta_eqn_expected_substituted = tsimplify(subs(theta_eqn_expected, subs_list))
thetat_eqn_expected_substituted = tsimplify(subs(thetat_eqn_expected, subs_list))
omega_eqn_expected_substituted = tsimplify(subs(omega_eqn_expected, subs_list))
omegat_eqn_expected_substituted = tsimplify(subs(omegat_eqn_expected, subs_list))
tau_eqn_expected_substituted = tsimplify(subs(tau_eqn_expected, subs_list))
taut_eqn_expected_substituted = tsimplify(subs(taut_eqn_expected, subs_list))
alpha_eqn_expected_substituted = tsimplify(subs(alpha_eqn_expected, subs_list))
alphat_eqn_expected_substituted = tsimplify(subs(alphat_eqn_expected, subs_list))
beta_eqn_expected_substituted = tsimplify(subs(beta_eqn_expected, subs_list))
betat_eqn_expected_substituted = tsimplify(subs(betat_eqn_expected, subs_list))

display(Equ(Symbol('\\text{Error}_{\\text{calculated vs expected ODEs}}^\\mu'), simplify(mu_eqn - mu_eqn_expected_substituted)))
display(Equ(Symbol('\\text{Error}_{\\text{calculated vs expected ODEs}}^{\\tilde{\\mu}}'), simplify(mut_eqn - mut_eqn_expected_substituted)))
display(Equ(Symbol('\\text{Error}_{\\text{calculated vs expected ODEs}}^\\nu'), simplify(nu_eqn - nu_eqn_expected_substituted)))
display(Equ(Symbol('\\text{Error}_{\\text{calculated vs expected ODEs}}^{\\tilde{\\nu}}'), simplify(nut_eqn - nut_eqn_expected_substituted)))
display(Equ(Symbol('\\text{Error}_{\\text{calculated vs expected ODEs}}^\\rho'), simplify(rho_eqn - rho_eqn_expected_substituted)))
display(Equ(Symbol('\\text{Error}_{\\text{calculated vs expected ODEs}}^{\\tilde{\\rho}}'), simplify(rhot_eqn - rhot_eqn_expected_substituted)))
display(Equ(Symbol('\\text{Error}_{\\text{calculated vs expected ODEs}}^\\theta'), simplify(theta_eqn - theta_eqn_expected_substituted)))
display(Equ(Symbol('\\text{Error}_{\\text{calculated vs expected ODEs}}^{\\tilde{\\theta}}'), simplify(thetat_eqn - thetat_eqn_expected_substituted)))
display(Equ(Symbol('\\text{Error}_{\\text{calculated vs expected ODEs}}^\\omega'), simplify(omega_eqn - omega_eqn_expected_substituted)))
display(Equ(Symbol('\\text{Error}_{\\text{calculated vs expected ODEs}}^{\\tilde{\\omega}}'), simplify(omegat_eqn - omegat_eqn_expected_substituted)))
display(Equ(Symbol('\\text{Error}_{\\text{calculated vs expected ODEs}}^\\tau'), simplify(tau_eqn - tau_eqn_expected_substituted)))
display(Equ(Symbol('\\text{Error}_{\\text{calculated vs expected ODEs}}^{\\tilde{\\tau}}'), simplify(taut_eqn - taut_eqn_expected_substituted)))
display(Equ(Symbol('\\text{Error}_{\\text{calculated vs expected ODEs}}^\\alpha'), simplify(alpha_eqn - alpha_eqn_expected_substituted)))
display(Equ(Symbol('\\text{Error}_{\\text{calculated vs expected ODEs}}^{\\tilde{\\alpha}}'), simplify(alphat_eqn - alphat_eqn_expected_substituted)))
display(Equ(Symbol('\\text{Error}_{\\text{calculated vs expected ODEs}}^\\beta'), simplify(beta_eqn - beta_eqn_expected_substituted)))
display(Equ(Symbol('\\text{Error}_{\\text{calculated vs expected ODEs}}^{\\tilde{\\beta}}'), simplify(betat_eqn - betat_eqn_expected_substituted)))

display(collect(subs(mu_eqn_expected, [(beta, epsilon**2 * beta), (alphat, epsilon**2 * alphat)]).expand() + O(epsilon**2), epsilon))
display(collect(subs(nu_eqn_expected, [(beta, epsilon**2 * beta), (alphat, epsilon**2 * alphat)]).expand() + O(epsilon**2), epsilon))
display(collect(subs(rho_eqn_expected, [(beta, epsilon**2 * beta), (alphat, epsilon**2 * alphat)]).expand() + O(epsilon**2), epsilon))
display(collect(subs(theta_eqn_expected, [(beta, epsilon**2 * beta), (alphat, epsilon**2 * alphat)]).expand() + O(epsilon**2), epsilon))
display(collect(subs(omega_eqn_expected, [(beta, epsilon**2 * beta), (alphat, epsilon**2 * alphat)]).expand() + O(epsilon**2), epsilon))
display(collect(subs(tau_eqn_expected, [(beta, epsilon**2 * beta), (alphat, epsilon**2 * alphat)]).expand() + O(epsilon**2), epsilon))
display(collect(subs(alpha_eqn_expected, [(beta, epsilon**2 * beta), (alphat, epsilon**2 * alphat)]).expand() + O(epsilon**2), epsilon))
display(collect(subs(beta_eqn_expected * epsilon**(-2), [(beta, epsilon**2 * beta), (alphat, epsilon**2 * alphat)]).expand() + O(epsilon**2), epsilon))

display(collect(subs(mut_eqn_expected, [(beta, epsilon**2 * beta), (alphat, epsilon**2 * alphat)]).expand() + O(epsilon**2), epsilon))
display(collect(subs(nut_eqn_expected, [(beta, epsilon**2 * beta), (alphat, epsilon**2 * alphat)]).expand() + O(epsilon**2), epsilon))
display(collect(subs(rhot_eqn_expected, [(beta, epsilon**2 * beta), (alphat, epsilon**2 * alphat)]).expand() + O(epsilon**2), epsilon))
display(collect(subs(thetat_eqn_expected, [(beta, epsilon**2 * beta), (alphat, epsilon**2 * alphat)]).expand() + O(epsilon**2), epsilon))
display(collect(subs(omegat_eqn_expected, [(beta, epsilon**2 * beta), (alphat, epsilon**2 * alphat)]).expand() + O(epsilon**2), epsilon))
display(collect(subs(taut_eqn_expected, [(beta, epsilon**2 * beta), (alphat, epsilon**2 * alphat)]).expand() + O(epsilon**2), epsilon))
display(collect(subs(alphat_eqn_expected * epsilon**(-2), [(beta, epsilon**2 * beta), (alphat, epsilon**2 * alphat)]).expand() + O(epsilon**2), epsilon))
display(collect(subs(betat_eqn_expected, [(beta, epsilon**2 * beta), (alphat, epsilon**2 * alphat)]).expand() + O(epsilon**2), epsilon))


Eq(\text{Error}_{\text{calculated vs expected ODEs}}^\mu, 0)

Eq(\text{Error}_{\text{calculated vs expected ODEs}}^{\tilde{\mu}}, 0)

Eq(\text{Error}_{\text{calculated vs expected ODEs}}^\nu, 0)

Eq(\text{Error}_{\text{calculated vs expected ODEs}}^{\tilde{\nu}}, 0)

Eq(\text{Error}_{\text{calculated vs expected ODEs}}^\rho, 0)

...

-2*I*\tilde{\theta}*z + 2*\tilde{\rho}*\tilde{\theta} - W_-/2 - W_+/2 - (\tilde{\theta})_{x} + sqrt(2)*(W_-)_{x}*\tilde{\tau}/2 + O(\epsilon**2)

-1 + 2*\tilde{\tau}**2*z**2 + 4*\tilde{\omega}*\tilde{\tau}*z**2 - W_-*\tilde{\tau}**2 - W_-*\tilde{\omega}**2 + sqrt(2)*(\tilde{\omega})_{x} + O(\epsilon**2)

1 + 2*\tilde{\tau}**2*z**2 - 2*W_-*\tilde{\omega}*\tilde{\tau} + sqrt(2)*(\tilde{\tau})_{x} + O(\epsilon**2)

4*\tilde{\alpha}*\tilde{\tau}*z**2 + 2*\tilde{\alpha}**2 - 2*W_-*\tilde{\tau}**2*z**2 - W_+ + sqrt(2)*(\tilde{\alpha})_{x} + O(\epsilon**2)

-2 + 4*\tilde{\omega}*\tilde{\tau}*z**2 - W_-*\tilde{\beta}**2 + sqrt(2)*(\tilde{\beta})_{x} + O(\epsilon**2)

# Expansions - General

In [11]:
def series_expansion(expr, x0, var, num):
    tvar = var.subs(z, tz).subs(lam, tlam)
    ser = complexify(series(realify(expr), x0=x0, x=tvar, n=num).removeO())
    expansion_dict = defaultdict(lambda: 0)
    for term in ser.as_ordered_terms():
        powers_dict = term.as_powers_dict()
        exponent = powers_dict.get(var, 0)
        coefficient = term.subs(var, 1)
        expansion_dict[exponent] += coefficient
    if not expansion_dict:
        return None, None
    final_dict = dict(expansion_dict)
    return final_dict

N = 7

sig_syms = D.create_diff_symbols(*[f'\\sigma_{{{n}}}' for n in range(0, N+1)])
sig_syms = sig_syms[1::2]
sig_syms = {n: sig_syms[n] for n in range(0, N+1)}
sigt_syms = D.create_diff_symbols(*[f'\\tilde{{\\sigma_{{{n}}}}}' for n in range(0, N+1)])
sigt_syms = sigt_syms[1::2]
sigt_syms = {n: sigt_syms[n] for n in range(0, N+1)}

sig_series = sum([s * (2 * I * z)**(-n) for n, s in sig_syms.items()])
sig_lam_series = sum([s * (2 * I * lam)**(-n) for n, s in sig_syms.items()])
sigt_series = sum([s * (2 * I * z)**(-n) for n, s in sigt_syms.items()])

r_syms = D.create_diff_symbols(*[f'r_{{{n}}}' for n in range(0, N+1)])
r_syms = r_syms[1::2]
r_syms = {n: r_syms[n] for n in range(0, N+1)}
R_syms = D.create_diff_symbols(*[f'R_{{{n}}}' for n in range(0, N+1)])
R_syms = R_syms[1::2]
R_syms = {n: R_syms[n] for n in range(0, N+1)}
rt_syms = D.create_diff_symbols(*[f'\\tilde{{r_{{{n}}}}}' for n in range(0, N+1)])
rt_syms = rt_syms[1::2]
rt_syms = {n: rt_syms[n] for n in range(0, N+1)}

R_series = sum([s * (2 * I * lam)**(-n) for n, s in r_syms.items()])
r_series = sum([s * (2 * I * z)**(-n) for n, s in r_syms.items()])
r_lam_series = sum([s * (2 * I * lam)**(-n) for n, s in r_syms.items()])
rt_series = sum([s * (2 * I * z)**(-n) for n, s in rt_syms.items()])

# Expansions - NLS

In [12]:
sig_expanded_NLS = {n: c / (2 * I)**n for n, c in series_expansion(sig_eqn_lam_NLS, oo, lam, N).items()}
sig_expanded_NLS = simplify(sum([(2 * I * lam)**n * c for n, c in sig_expanded_NLS.items()])).expand()
sig_expanded_NLS = simplify(subs(sig_expanded_NLS, [(sig, sig_lam_series), (r, r_lam_series)])).expand()
sig_orders_NLS = {n: c / (2 * I)**n for n, c in series_expansion(sig_expanded_NLS, oo, lam, N).items()}
sig_orders_NLS = {n: sig_orders_NLS[n] for n in sorted(sig_orders_NLS, reverse=True)}
sig_iter_NLS = {1-n: solve(e, sig_syms[1-n])[-1] for n, e in sig_orders_NLS.items()}

r_eqn_lam_NLS = r_eqn_NLS.subs(z, z_NLS)
r_expanded_NLS = {n: c / (2 * I)**n for n, c in series_expansion(r_eqn_lam_NLS, oo, lam, N).items()}
r_expanded_NLS = simplify(sum([(2 * I * lam)**n * c for n, c in r_expanded_NLS.items()])).expand()
r_expanded_NLS = simplify(subs(r_expanded_NLS, [(sig, sig_lam_series), (r, r_lam_series)])).expand()
r_orders_NLS = {n: c / (2 * I)**n for n, c in series_expansion(r_expanded_NLS, oo, lam, N).items()}
r_orders_NLS = {n: r_orders_NLS[n] for n in sorted(r_orders_NLS, reverse=True)}
r_iter_NLS = {1-n: solve(e, r_syms[1-n])[-1] for n, e in r_orders_NLS.items()}

sig_expr_NLS = {n: 0 for n in sig_iter_NLS.keys()}
r_expr_NLS = {n: 0 for n in r_iter_NLS.keys()}

for n in sig_iter_NLS.keys():
    print("NLS n =", n)
    sig_subs_list = [(sig_syms[k], sig_expr_NLS[k]) for k in sig_expr_NLS.keys() if k < n]
    r_subs_list = [(r_syms[k], r_expr_NLS[k]) for k in r_expr_NLS.keys() if k < n]
    subs_list = [sub for sub_pair in zip(sig_subs_list, r_subs_list) for sub in sub_pair]
    
    sig_expr_NLS[n] = simplify(subs(sig_iter_NLS[n], subs_list))
    r_expr_NLS[n] = simplify(subs(r_iter_NLS[n], subs_list))
        
s = 1 #1: correct sign, -1: wrong sign (old)

r_eq_NLS = [r_syms[n] for n in range(N+1)]
sig_eq_NLS = [sig_syms[n] for n in range(N+1)]

sig_eq_NLS[0] = 0
sig_eq_NLS[1] = - q * q_conj

r_eq_NLS[0] = 0
r_eq_NLS[1] = - s * q_conj

for n in range(1, N):
    print("n =", n)

    X = s * deriv(r_eq_NLS[n])
    for k in range(0, n + 1):
        X += s * q * (r_eq_NLS[k] * r_eq_NLS[n-k])
    r_eq_NLS[n+1] = simplify(X)

    X = deriv(sig_eq_NLS[n])
    for k in range(0, n + 1):
        X += sig_eq_NLS[k] * sig_eq_NLS[n-k]
    X += - deriv(q) * r_eq_NLS[n]
    sig_eq_NLS[n+1] = simplify(X)

for n in range(0, N+1):
    display(Equ(Symbol('\\text{Error}_{\\text{NLS $r$ symbolic vs iteration}}'), simplify(r_eq_NLS[n] - r_expr_NLS[n])))
    display(Equ(Symbol('\\text{Error}_{\\text{NLS $\sigma$ symbolic vs iteration}}'), simplify(sig_eq_NLS[n] - sig_expr_NLS[n])))


NLS n = 0
NLS n = 1
NLS n = 2
NLS n = 3
NLS n = 4
NLS n = 5
NLS n = 6
NLS n = 7
n = 1
n = 2
n = 3
n = 4
n = 5
n = 6


Eq(\text{Error}_{\text{NLS $r$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{NLS $\sigma$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{NLS $r$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{NLS $\sigma$ symbolic vs iteration}}, 0)

...

Eq(\text{Error}_{\text{NLS $\sigma$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{NLS $r$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{NLS $\sigma$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{NLS $r$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{NLS $\sigma$ symbolic vs iteration}}, 0)

# Expansions - GP

In [13]:
sig_expanded_GP = {n: c / (2 * I)**n for n, c in series_expansion(sig_eqn_GP, oo, z, N).items()}
sig_expanded_GP = simplify(sum([(2 * I * z)**n * c for n, c in sig_expanded_GP.items()])).expand()
sig_expanded_GP = simplify(subs(sig_expanded_GP, [(sig, sig_series), (r, r_series)])).expand()
sig_orders_GP = {n: c / (2 * I)**n for n, c in series_expansion(sig_expanded_GP, oo, z, N).items()}
sig_orders_GP = {n: sig_orders_GP[n] for n in sorted(sig_orders_GP, reverse=True)}
sig_iter_GP = {1-n: solve(e, sig_syms[1-n])[-1] for n, e in sig_orders_GP.items()}

r_expanded_GP = {n: c / (2 * I)**n for n, c in series_expansion(r_eqn_GP, oo, z, N).items()}
r_expanded_GP = simplify(sum([(2 * I * z)**n * c for n, c in r_expanded_GP.items()])).expand()
r_expanded_GP = simplify(subs(r_expanded_GP, [(sig, sig_series), (r, r_series)])).expand()
r_orders_GP = {n: c / (2 * I)**n for n, c in series_expansion(r_expanded_GP, oo, z, N).items()}
r_orders_GP = {n: r_orders_GP[n] for n in sorted(r_orders_GP, reverse=True)}
r_iter_GP = {1-n: solve(e, r_syms[1-n])[-1] for n, e in r_orders_GP.items()}

R_expanded_GP = {n: c / (2 * I)**n for n, c in series_expansion(r_eqn_lam_GP, oo, lam, N).items()}
R_expanded_GP = simplify(sum([(2 * I * lam)**n * c for n, c in R_expanded_GP.items()])).expand()
R_expanded_GP = simplify(subs(R_expanded_GP, [(sig, sig_series), (r, r_lam_series)])).expand()
R_orders_GP = {n: c / (2 * I)**n for n, c in series_expansion(R_expanded_GP, oo, lam, N).items()}
R_orders_GP = {n: R_orders_GP[n] for n in sorted(R_orders_GP, reverse=True)}
R_iter_GP = {1-n: solve(e, r_syms[1-n])[-1] for n, e in R_orders_GP.items()}

sig_expr_GP = {n: 0 for n in sig_iter_GP.keys()}
r_expr_GP = {n: 0 for n in r_iter_GP.keys()}
R_expr_GP = {n: 0 for n in R_iter_GP.keys()}

for n in sig_iter_GP.keys():
    print("GP n =", n)
    sig_subs_list = [(sig_syms[k], sig_expr_GP[k]) for k in sig_expr_GP.keys() if k < n]
    r_subs_list = [(r_syms[k], r_expr_GP[k]) for k in r_expr_GP.keys() if k < n]
    R_subs_list = [(r_syms[k], R_expr_GP[k]) for k in r_expr_GP.keys() if k < n]
    subs_list = [sub for sub_pair in zip(sig_subs_list, r_subs_list) for sub in sub_pair]
    
    sig_expr_GP[n] = simplify(subs(sig_iter_GP[n], subs_list))
    r_expr_GP[n] = simplify(subs(r_iter_GP[n], subs_list))
    R_expr_GP[n] = simplify(subs(R_iter_GP[n], R_subs_list))
    
s = 1 #1: correct sign, -1: wrong sign (old)

sig_eq_GP = [sig_syms[n] for n in range(N+1)]
r_eq_GP = [r_syms[n] for n in range(N+1)]

sig_eq_GP[0] = 0
sig_eq_GP[1] = 1 - q * q_conj

r_eq_GP[0] = 0
r_eq_GP[1] = - s * q_conj

for n in range(1, N):
    print("n =", n)
    X = 0
    for m in range(0, n//2 + 1):
        X += s * deriv(r_eq_GP[n-2*m]) * binomial(m - Rational(n,2), m) * 4**m
    for k in range(0, n + 1):
        for m1 in range(0, k//2 + 1):
            for m2 in range(0, (n-k)//2 + 1):
                X += s * q * r_eq_GP[k-2*m1] * binomial(m1 - Rational(k,2), m1) * 4**m1 * r_eq_GP[n-k-2*m2] * binomial(m2 - Rational(n-k,2), m2) * 4**m2
    for m in range(1, (n+1)//2 + 1):
        X += - r_eq_GP[n+1-2*m] * binomial(m - Rational(n+1,2), m) * 4**m
    r_eq_GP[n+1] = simplify(X)
    
    X = deriv(sig_eq_GP[n])
    for k in range(0, n + 1):
        X += sig_eq_GP[k] * sig_eq_GP[n-k]  
    X += - deriv(q) * r_eq_GP[n]
    sig_eq_GP[n+1] = simplify(X)
    
for n in range(0, N+1):
    display(Equ(Symbol('\\text{Error}_{\\text{GP $r$ symbolic vs iteration}}'), simplify(r_eq_GP[n] - r_expr_GP[n])))
    display(Equ(Symbol('\\text{Error}_{\\text{GP $\sigma$ symbolic vs iteration}}'), simplify(sig_eq_GP[n] - sig_expr_GP[n])))


GP n = 0
GP n = 1
GP n = 2
GP n = 3
GP n = 4
GP n = 5
GP n = 6
GP n = 7
n = 1
n = 2
n = 3
n = 4
n = 5
n = 6


Eq(\text{Error}_{\text{GP $r$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{GP $\sigma$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{GP $r$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{GP $\sigma$ symbolic vs iteration}}, 0)

...

Eq(\text{Error}_{\text{GP $\sigma$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{GP $r$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{GP $\sigma$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{GP $r$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{GP $\sigma$ symbolic vs iteration}}, 0)

# Expansions - hGP

In [14]:
sig_expanded_hGP = {n: c / (2 * I)**n for n, c in series_expansion(sig_eqn_hGP, oo, z, N).items()}
sig_expanded_hGP = simplify(sum([(2 * I * z)**n * c for n, c in sig_expanded_hGP.items()])).expand()
sig_expanded_hGP = simplify(subs(sig_expanded_hGP, [(sig, sig_series), (r, r_series)])).expand()
sig_orders_hGP = {n: c / (2 * I)**n for n, c in series_expansion(sig_expanded_hGP, oo, z, N).items()}
sig_orders_hGP = {n: sig_orders_hGP[n] for n in sorted(sig_orders_hGP, reverse=True)}
sig_iter_hGP = {1-n: solve(e, sig_syms[1-n])[-1] for n, e in sig_orders_hGP.items()}

r_eqn_lam_hGP = r_eqn_hGP.subs(z, z_hGP)
r_expanded_hGP = {n: c / (2 * I)**n for n, c in series_expansion(r_eqn_hGP, oo, z, N).items()}
r_expanded_hGP = simplify(sum([(2 * I * z)**n * c for n, c in r_expanded_hGP.items()])).expand()
r_expanded_hGP = simplify(subs(r_expanded_hGP, [(sig, sig_series), (r, r_series)])).expand()
r_orders_hGP = {n: c / (2 * I)**n for n, c in series_expansion(r_expanded_hGP, oo, z, N).items()}
r_orders_hGP = {n: r_orders_hGP[n] for n in sorted(r_orders_hGP, reverse=True)}
r_iter_hGP = {1-n: solve(e, r_syms[1-n])[-1] for n, e in r_orders_hGP.items()}

sig_expr_hGP = {n: 0 for n in sig_iter_hGP.keys()}
r_expr_hGP = {n: 0 for n in r_iter_hGP.keys()}

for n in sig_iter_hGP.keys():
    print("hGP n =", n)
    sig_subs_list = [(sig_syms[k], sig_expr_hGP[k]) for k in sig_expr_hGP.keys() if k < n]
    r_subs_list = [(r_syms[k], r_expr_hGP[k]) for k in r_expr_hGP.keys() if k < n]
    subs_list = [sub for sub_pair in zip(sig_subs_list, r_subs_list) for sub in sub_pair]
    
    sig_expr_hGP[n] = simplify(subs(sig_iter_hGP[n], subs_list))
    r_expr_hGP[n] = simplify(subs(r_iter_hGP[n], subs_list))
    

def qt_coeff(n):
    if n == -1:
        return 1 / (I * 2)
    if n < -1 or (n % 2 == 0 and n >= 0):
        return 0
    else:
        return I * catalan((n - 1) // 2)


r_eq_hGP = [r_syms[n] for n in range(N+1)]
r_factor = 1
r_eq_hGP[0] = r_factor
r_eq_hGP[1] = r_factor / I * (w_plus - w_minus + 2)

sig_eq_hGP = [sig_syms[n] for n in range(N+1)]
sig_eq_hGP[0] = - I / 2 * (w_plus + w_minus)
sig_eq_hGP[1] = I / 2 * deriv(w_plus - w_minus) - (w_plus - w_minus)**2 / 4 - (w_plus - w_minus)

for n in range(1, N):
    print("Iter rGP n =", n)
    X = deriv(r_eq_hGP[n])
    X += qt_coeff(n) * I
    for k in range(0, n+1):
        X += (- w_plus - 1) * r_eq_hGP[n-k] * r_eq_hGP[k] * I
    for m in range(0, n+1):
        for k in range(0, m+1):     
            X += - qt_coeff(n-m) * r_eq_hGP[m-k] * r_eq_hGP[k] * I
    for k in range(1, n+1):
        X += - qt_coeff(-1) * r_eq_hGP[n+1-k] * r_eq_hGP[k] * I
    r_eq_hGP[n+1] = simplify(r_factor * X)
    
    X = deriv(sig_eq_hGP[n])
    for k in range(0, n+1):
        X += sig_eq_hGP[n-k] * sig_eq_hGP[k]
    X += qt_coeff(n) * (w_minus + w_plus)
    X += deriv(w_plus) * r_eq_hGP[n] * I
    sig_eq_hGP[n+1] = simplify(X)
    
for n in range(0, N+1):
    display(Equ(Symbol('\\text{Error}_{\\text{hGP $r$ symbolic vs iteration}}'), simplify(r_eq_hGP[n] - r_expr_hGP[n])))
    display(Equ(Symbol('\\text{Error}_{\\text{hGP $\sigma$ symbolic vs iteration}}'), simplify(sig_eq_hGP[n] - sig_expr_hGP[n])))


hGP n = 0
hGP n = 1
hGP n = 2
hGP n = 3
hGP n = 4
hGP n = 5
hGP n = 6
hGP n = 7
Iter rGP n = 1
Iter rGP n = 2
Iter rGP n = 3
Iter rGP n = 4
Iter rGP n = 5
Iter rGP n = 6


Eq(\text{Error}_{\text{hGP $r$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{hGP $\sigma$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{hGP $r$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{hGP $\sigma$ symbolic vs iteration}}, 0)

...

Eq(\text{Error}_{\text{hGP $\sigma$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{hGP $r$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{hGP $\sigma$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{hGP $r$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{hGP $\sigma$ symbolic vs iteration}}, 0)

# Expansions - rGP

In [15]:
sig_expanded_rGP = {n: c / (2 * I)**n for n, c in series_expansion(sig_eqn_rGP, oo, z, N).items()}
sig_expanded_rGP = simplify(sum([(2 * I * z)**n * c for n, c in sig_expanded_rGP.items()])).expand()
sig_expanded_rGP = simplify(subs(sig_expanded_rGP, [(sig, sig_series), (r, r_series)])).expand()
sig_orders_rGP = {n: c / (2 * I)**n for n, c in series_expansion(sig_expanded_rGP, oo, z, N).items()}
sig_orders_rGP = {n: sig_orders_rGP[n] for n in sorted(sig_orders_rGP, reverse=True)}
sig_iter_rGP = {1-n: solve(e, sig_syms[1-n])[-1] for n, e in sig_orders_rGP.items()}

sigt_expanded_rGP = {n: c / (2 * I)**n for n, c in series_expansion(sigt_eqn_rGP, oo, z, N).items()}
sigt_expanded_rGP = simplify(sum([(2 * I * z)**n * c for n, c in sigt_expanded_rGP.items()])).expand()
sigt_expanded_rGP = simplify(subs(sigt_expanded_rGP, [(sigt, sigt_series), (rt, rt_series)])).expand()
sigt_orders_rGP = {n: c / (2 * I)**n for n, c in series_expansion(sigt_expanded_rGP, oo, z, N).items()}
sigt_orders_rGP = {n: sigt_orders_rGP[n] for n in sorted(sigt_orders_rGP, reverse=True)}
sigt_iter_rGP = {1-n: solve(e, sigt_syms[1-n])[-1] for n, e in sigt_orders_rGP.items()}

r_expanded_rGP = {n: c / (2 * I)**n for n, c in series_expansion(r_eqn_rGP, oo, z, N).items()}
r_expanded_rGP = simplify(sum([(2 * I * z)**n * c for n, c in r_expanded_rGP.items()])).expand()
r_expanded_rGP = simplify(subs(r_expanded_rGP, [(sig, sig_series), (r, r_series)])).expand()
r_orders_rGP = {n: c / (2 * I)**n for n, c in series_expansion(r_expanded_rGP, oo, z, N).items()}
r_orders_rGP = {n: r_orders_rGP[n] for n in sorted(r_orders_rGP, reverse=True)}
r_iter_rGP = {1-n: solve(e, r_syms[1-n])[-1] for n, e in r_orders_rGP.items()}

rt_expanded_rGP = {n: c / (2 * I)**n for n, c in series_expansion(rt_eqn_rGP, oo, z, N).items()}
rt_expanded_rGP = simplify(sum([(2 * I * z)**n * c for n, c in rt_expanded_rGP.items()])).expand()
rt_expanded_rGP = simplify(subs(rt_expanded_rGP, [(sigt, sigt_series), (rt, rt_series)])).expand()
rt_orders_rGP = {n: c / (2 * I)**n for n, c in series_expansion(rt_expanded_rGP, oo, z, N).items()}
rt_orders_rGP = {n: rt_orders_rGP[n] for n in sorted(rt_orders_rGP, reverse=True)}
rt_iter_rGP = {1-n: solve(e, rt_syms[1-n])[-1] for n, e in rt_orders_rGP.items()}

# Not necessary it seems
# sig_iter_rGP = {k: sig_iter_rGP.get(k, 0) for k in set(sig_iter_rGP.keys()) | set(r_iter_rGP.keys())}
# r_iter_rGP = {k: r_iter_rGP.get(k, 0) for k in set(sig_iter_rGP.keys()) | set(r_iter_rGP.keys())}

sig_expr_rGP = {n: 0 for n in sig_iter_rGP.keys()}
sigt_expr_rGP = {n: 0 for n in sigt_iter_rGP.keys()}
r_expr_rGP = {n: 0 for n in r_iter_rGP.keys()}
rt_expr_rGP = {n: 0 for n in rt_iter_rGP.keys()}


for n in sig_iter_rGP.keys():
    print("rGP n =", n)
    sig_subs_list = [(sig_syms[k], sig_expr_rGP[k]) for k in sig_expr_rGP.keys() if k < n]
    r_subs_list = [(r_syms[k], r_expr_rGP[k]) for k in r_expr_rGP.keys() if k < n]
    subs_list = [sub for sub_pair in zip(sig_subs_list, r_subs_list) for sub in sub_pair]
    
    sig_expr_rGP[n] = simplify(subs(sig_iter_rGP[n], subs_list))
    r_expr_rGP[n] = simplify(subs(r_iter_rGP[n], subs_list))

    sigt_subs_list = [(sigt_syms[k], sigt_expr_rGP[k]) for k in sigt_expr_rGP.keys() if k < n]
    rt_subs_list = [(rt_syms[k], rt_expr_rGP[k]) for k in rt_expr_rGP.keys() if k < n]
    subs_list = [sub for sub_pair in zip(sigt_subs_list, rt_subs_list) for sub in sub_pair]
    
    sigt_expr_rGP[n] = simplify(subs(sigt_iter_rGP[n], subs_list))    
    rt_expr_rGP[n] = simplify(subs(rt_iter_rGP[n], subs_list))


def q_coeff(n):
    if n == -1:
        return epsilon / (I * sqrt(2))
    if n < -1 or (n % 2 == 0 and n >= 0):
        return 0
    else:
        return I * catalan((n - 1) // 2) / sqrt(2)**n / epsilon**n

r_eq = [r_syms[n] for n in range(N+1)]
r_factor = 1
r_eq[0] = r_factor
r_eq[1] = r_factor / I * epsilon / sqrt(2) * (W_plus - W_minus + 2 / epsilon**2)

sig_eq = [sig_syms[n] for n in range(N+1)]
sig_eq[0] = q_coeff(-1) / 2 * (W_plus + W_minus)
sig_eq[1] = q_coeff(-1) / 2 * deriv(W_plus + W_minus) + q_coeff(-1)**2 / 4 * (W_plus + W_minus)**2 
sig_eq[1] += epsilon**2 / 2 * W_minus * W_plus + (W_minus - W_plus) / 2 
sig_eq[1] += epsilon / sqrt(2) * deriv(W_plus) * r_eq[0]  * I

for n in range(1, N):
    print("Iter rGP n =", n)
    X = deriv(r_eq[n])
    X += - q_coeff(n) / sqrt(2) / epsilon / I
    for k in range(0, n+1):
        X += epsilon / sqrt(2) * (- W_plus - 1 / epsilon**2) * r_eq[n-k] * r_eq[k] * I
    for m in range(0, n+1):
        for k in range(0, m+1):     
            X += - 1 / epsilon / sqrt(2) * q_coeff(n-m) * r_eq[m-k] * r_eq[k] * I
    for k in range(1, n+1):
        X += - q_coeff(-1) / sqrt(2) / epsilon * r_eq[n+1-k] * r_eq[k] * I
    r_eq[n+1] = simplify(r_factor * X)
    
    X = deriv(sig_eq[n])
    for k in range(0, n+1):
        X += sig_eq[n-k] * sig_eq[k]
    X += q_coeff(n) / 2 * (W_minus + W_plus)
    X += epsilon / sqrt(2) * deriv(W_plus) * r_eq[n] * I
    sig_eq[n+1] = simplify(X)
    
for n in range(0, N+1):
    display(Equ(Symbol('\\text{Error}_{\\text{rGP $r$ symbolic vs iteration}}'), simplify(r_eq[n] - r_expr_rGP[n])))
    display(Equ(Symbol('\\text{Error}_{\\text{rGP $\sigma$ symbolic vs iteration}}'), simplify(sig_eq[n] - sig_expr_rGP[n])))


rGP n = 0
rGP n = 1
rGP n = 2
rGP n = 3
rGP n = 4
rGP n = 5
rGP n = 6
rGP n = 7
Iter rGP n = 1
Iter rGP n = 2
Iter rGP n = 3
Iter rGP n = 4
Iter rGP n = 5
Iter rGP n = 6


Eq(\text{Error}_{\text{rGP $r$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{rGP $\sigma$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{rGP $r$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{rGP $\sigma$ symbolic vs iteration}}, 0)

...

Eq(\text{Error}_{\text{rGP $\sigma$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{rGP $r$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{rGP $\sigma$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{rGP $r$ symbolic vs iteration}}, 0)

Eq(\text{Error}_{\text{rGP $\sigma$ symbolic vs iteration}}, 0)

In [ ]:
for n in range(N+1):
    display(Equ(sig_syms[n], variation(sig_expr_rGP[n] - sigt_expr_rGP[n], W_plus)))
    display(Equ(sig_syms[n], variation(sig_expr_rGP[n] - sigt_expr_rGP[n], W_minus)))


# Expansions - KdV

In [16]:
N = 7
sig_eq_plus_KdV = [sig_syms[n] for n in range(N+1)]
sig_eq_minus_KdV = [sig_syms[n] for n in range(N+1)]
sig_eq_plus_KdV[0] = 0
sig_eq_minus_KdV[0] = 0
sig_eq_plus_KdV[1] = - W_plus
sig_eq_minus_KdV[1] = W_minus

for n in range(1, N):
    print("Iter KdV n =", n)

    X = deriv(sig_eq_plus_KdV[n])
    Y = deriv(sig_eq_minus_KdV[n])
    for k in range(0, n+1):
        X += sig_eq_plus_KdV[n-k] * sig_eq_plus_KdV[k]
        Y += sig_eq_minus_KdV[n-k] * sig_eq_minus_KdV[k]

    sig_eq_plus_KdV[n+1] = simplify(X)
    sig_eq_minus_KdV[n+1] = simplify(Y)


Iter KdV n = 1
Iter KdV n = 2
Iter KdV n = 3
Iter KdV n = 4
Iter KdV n = 5
Iter KdV n = 6


In [ ]:
# for n in range(8):
#     print("n =", n)
#     display(simplify(sig_eq_minus_KdV[n]))
#     display(simplify(sig_eq_plus_KdV[n]))

for n in range(8):
    #display(simplify(Rational(1,2) * deriv(variation(sig_eq_minus_KdV[n], W_minus))))
    display(simplify(Rational(1,2) * deriv(variation(sig_eq_plus_KdV[n], W_plus))))

# Hamiltonians

In [17]:
N = 7

In [18]:
H_NLS = {n-1: - (-I)**(n-1) * e for n, e in sig_expr_NLS.items()}
H_NLS[-1] += simplify(- (-I)**(-1) * T_boundary_NLS.subs(phi_plus, 0).subs(phi_minus, 0))

H_GP = {n-1: - (-I)**(n-1) * e for n, e in sig_expr_GP.items()}
H_GP[-1] += simplify(- (-I)**(-1) * T_boundary_GP.subs(phi_plus, (q_conj * deriv(q) - q * deriv(q_conj)) / (2 * I * q * q_conj)).subs(phi_minus, 0))

H_hGP = {n-1: - (-I)**(n-1) * e for n, e in sig_expr_hGP.items()}
H_hGP[-1] += simplify(- (-I)**(-1) * T_boundary_hGP.subs(phi_plus, w_plus + w_minus).subs(phi_minus, 0))

H_rGP = {n-1: - (-I)**(n-1) * (sqrt(2) * epsilon)**n * e for n, e in sig_expr_rGP.items()}
H_rGP[-1] += simplify(- (-I)**(-1) * (sqrt(2) * epsilon)**(-1) * T_boundary_rGP.subs(phi_plus, epsilon**2 * (W_plus + W_minus)).subs(phi_minus, 0))

Ht_rGP = {n-1: - (-I)**(n-1) * (sqrt(2) * epsilon)**n * e for n, e in sigt_expr_rGP.items()}
Ht_rGP[-1] += simplify(- (-I)**(-1) * (sqrt(2) * epsilon)**(-1) * T_boundary_rGP.subs(phi_plus, epsilon**2 * (W_plus + W_minus)).subs(phi_minus, 0))

H_wpGP = {n: 0 for n in range(len(H_rGP.keys())) if 2*n in H_rGP.keys() and 2*n-1 in H_rGP.keys()}
H_wmGP = {n: 0 for n in range(len(H_rGP.keys())) if 2*n in H_rGP.keys() and 2*n-1 in H_rGP.keys()}

for n in range(0, N//2+1):
    X = re(H_rGP[2*n])
    Y = re(Ht_rGP[2*n])
    for j in range(n+1):
        expansion_coeff = binomial(-Rational(1,2), n-j) * 4**(n-j) * (2 - (1 if j == 0 else 0))
        X += expansion_coeff * re(H_rGP[2*j-1])
        Y -= expansion_coeff * re(Ht_rGP[2*j-1])
    H_wmGP[n] = collect(simplify(X).expand(), epsilon)
    H_wpGP[n] = collect(simplify(Y).expand(), epsilon)
    
H_plus_KdV = {n-1: (-1)**(n+1) * sig_eq_plus_KdV[2*n+1] for n in range((len(sig_eq_plus_KdV) - 1) // 2 + 1)}
H_minus_KdV = {n-1: (-1)**(n+1) * sig_eq_minus_KdV[2*n+1] for n in range((len(sig_eq_minus_KdV) - 1) // 2 + 1)}


In [19]:
for n in H_wpGP.keys():
    display(Equ(Symbol(f'\\text{{Error}}_{{\\text{{wpGP vs KdV}}}}^{{{n}}}'), collect(simplify(
        1 / (sqrt(2) * epsilon)**(2*n-1) / 4 * H_wpGP[n] - epsilon**2 / 2 * H_plus_KdV[n-1] + O(epsilon**4)
    ).expand().removeO(), epsilon)))
    display(Equ(Symbol(f'\\text{{Error}}_{{\\text{{mpGP vs KdV}}}}^{{{n}}}'), collect(simplify(
        1 / (sqrt(2) * epsilon)**(2*n-1) / 4 * H_wmGP[n] - epsilon**2 / 2 * H_minus_KdV[n-1] + O(epsilon**4)
    ).expand().removeO(), epsilon)))


Eq(\text{Error}_{\text{wpGP vs KdV}}^{0}, 0)

Eq(\text{Error}_{\text{mpGP vs KdV}}^{0}, 0)

Eq(\text{Error}_{\text{wpGP vs KdV}}^{1}, 0)

Eq(\text{Error}_{\text{mpGP vs KdV}}^{1}, 0)

Eq(\text{Error}_{\text{wpGP vs KdV}}^{2}, 0)

Eq(\text{Error}_{\text{mpGP vs KdV}}^{2}, 0)

Eq(\text{Error}_{\text{wpGP vs KdV}}^{3}, 0)

Eq(\text{Error}_{\text{mpGP vs KdV}}^{3}, 0)

# NLS Hierarchy

In [21]:
T_list = {n: T for n in range(-1, N+1) for TT, T in [D.create_non_diff_symbols(f'T_{{{n}}}', real=True)]}
t_list = {n: t for n in range(-1, N+1) for tt, t in [D.create_non_diff_symbols(f't_{{{n}}}', real=True)]}
X = D.create_non_diff_symbols('X', real=True)


In [22]:
J_NLS = I * Matrix([[0, - 1], [1, 0]])

NLS_hierarchy_full_LHS = {n: Matrix([multi_deriv(q, T_list[n]), multi_deriv(q_conj, T_list[n])]) for n in H_NLS.keys()}
NLS_hierarchy_full_RHS = {n: evaluateExpr(J_NLS * D.comm_to_non_comm(Matrix([variation(H_NLS[n], q), variation(H_NLS[n], q_conj)]))) for n in H_NLS.keys()}

# GP Hierarchy

In [23]:
J_GP = I * Matrix([[0, - 1], [1, 0]])

GP_hierarchy_full_LHS = {n: Matrix([multi_deriv(q, T_list[n]), multi_deriv(q_conj, T_list[n])]) for n in H_GP.keys()}
GP_hierarchy_full_RHS = {n: evaluateExpr(J_GP * D.comm_to_non_comm(Matrix([variation(H_GP[n], q), variation(H_GP[n], q_conj)]))) for n in H_GP.keys()}

# hGP Hierarchy

In [24]:
J_hGP = Matrix([
    [- D(xx) / aa - 1 / aa * D(xx), D(xx) / aa - 1 / aa * D(xx)],
    [- D(xx) / aa + 1 / aa * D(xx), D(xx) / aa + 1 / aa * D(xx)]
]) / 4

hGP_hierarchy_full_LHS = {n: Matrix([multi_deriv(w_plus, T_list[n]), multi_deriv(w_minus, T_list[n])]) for n in H_hGP.keys()}
hGP_hierarchy_full_RHS = {n: evaluateExpr(J_hGP * D.comm_to_non_comm(Matrix([variation(H_hGP[n], w_plus), variation(H_hGP[n], w_minus)]))) for n in H_hGP.keys()}

# rGP Hierarchy

In [25]:
J_rGP = Matrix([
    [- D(xx) / AA - 1 / AA * D(xx), D(xx) / AA - 1 / AA * D(xx)],
    [- D(xx) / AA + 1 / AA * D(xx), D(xx) / AA + 1 / AA * D(xx)]
]) / (2 * epsilon**2)

rGP_hierarchy_full_LHS = {n: Matrix([multi_deriv(W_plus, T_list[n]), multi_deriv(W_minus, T_list[n])]) for n in H_rGP.keys()}
rGP_hierarchy_full_RHS = {n: evaluateExpr(J_rGP * D.comm_to_non_comm(Matrix([variation(H_rGP[n], W_plus), variation(H_rGP[n], W_minus)]))) for n in H_rGP.keys()}

# wpGM and wmGP Hierarchy

In [26]:
J_wpmGP = Matrix([
    [- D(xx) - D(xx), D(xx) - D(xx)],
    [- D(xx) + D(xx), D(xx) + D(xx)]
]) / (2 * epsilon**2)

wpGP_hierarchy_full_LHS = {n: Matrix([multi_deriv(W_plus, T_list[n]), multi_deriv(W_minus, T_list[n])]) for n in H_wpGP.keys()}
wpGP_hierarchy_full_RHS = {n: evaluateExpr(1 / (sqrt(2) * epsilon)**(2*n-1) / 4 * J_wpmGP * D.comm_to_non_comm(Matrix([variation(H_wpGP[n], W_plus), variation(H_wpGP[n], W_minus)]))) for n in H_wpGP.keys()}
wmGP_hierarchy_full_LHS = {n: Matrix([multi_deriv(W_plus, T_list[n]), multi_deriv(W_minus, T_list[n])]) for n in H_wmGP.keys()}
wmGP_hierarchy_full_RHS = {n: evaluateExpr(1 / (sqrt(2) * epsilon)**(2*n-1) / 4 * J_wpmGP * D.comm_to_non_comm(Matrix([variation(H_wmGP[n], W_plus), variation(H_wmGP[n], W_minus)]))) for n in H_wmGP.keys()}

# KdV Hierarchy

In [27]:
J_KdV = Rational(1,2) * D(xx)

KdV_hierarchy_plus_LHS = {n: multi_deriv(W_plus, T_list[n]) for n in H_plus_KdV.keys()}
KdV_hierarchy_plus_RHS = {n: evaluateExpr(J_KdV * D.comm_to_non_comm(variation(H_plus_KdV[n], W_plus))) for n in H_plus_KdV.keys()}

KdV_hierarchy_minus_LHS = {n: multi_deriv(W_minus, T_list[n]) for n in H_minus_KdV.keys()}
KdV_hierarchy_minus_RHS = {n: evaluateExpr(J_KdV * D.comm_to_non_comm(variation(H_minus_KdV[n], W_minus))) for n in H_minus_KdV.keys()}


In [ ]:
display(multi_deriv(u, T_list[5]))

In [32]:
qq_overline, q_overline = D.create_diff_symbols('\\overline{q}', real=False)
for n in GP_hierarchy_full_RHS.keys():
    res = Equ(multi_deriv(I * q, t_list[n]), simplify(subs(I * GP_hierarchy_full_RHS[n][0], [(q_conj, q_overline)])))
    display(res)

Eq(I*(q)_{t_{-1}}, 0)

Eq(I*(q)_{t_{0}}, q)

Eq(I*(q)_{t_{1}}, I*(q)_{x})

Eq(I*(q)_{t_{2}}, -(q)_{xx} + 2*\overline{q}*q**2 - 2*q)

Eq(I*(q)_{t_{3}}, I*(-(q)_{xxx} + 6*(q)_{x}*\overline{q}*q - 4*(q)_{x}))

Eq(I*(q)_{t_{4}}, -2*(\overline{q})_{xx}*q**2 - 4*(\overline{q})_{x}*(q)_{x}*q + (q)_{xxxx} - 8*(q)_{xx}*\overline{q}*q + 6*(q)_{xx} - 6*(q)_{x}**2*\overline{q} + 6*\overline{q}**2*q**3 - 12*\overline{q}*q**2 + 6*q)

Eq(I*(q)_{t_{5}}, I*(-10*(\overline{q})_{xx}*(q)_{x}*q - 10*(\overline{q})_{x}*(q)_{xx}*q - 10*(\overline{q})_{x}*(q)_{x}**2 + (q)_{xxxxx} - 10*(q)_{xxx}*\overline{q}*q + 8*(q)_{xxx} - 20*(q)_{xx}*(q)_{x}*\overline{q} + 30*(q)_{x}*\overline{q}**2*q**2 - 48*(q)_{x}*\overline{q}*q + 16*(q)_{x}))

Eq(I*(q)_{t_{6}}, 2*(\overline{q})_{xxxx}*q**2 + 8*(\overline{q})_{xxx}*(q)_{x}*q + 22*(\overline{q})_{xx}*(q)_{xx}*q + 20*(\overline{q})_{xx}*(q)_{x}**2 - 20*(\overline{q})_{xx}*\overline{q}*q**3 + 20*(\overline{q})_{xx}*q**2 - 10*(\overline{q})_{x}**2*q**3 + 18*(\overline{q})_{x}*(q)_{xxx}*q + 50*(\overline{q})_{x}*(q)_{xx}*(q)_{x} - 60*(\overline{q})_{x}*(q)_{x}*\overline{q}*q**2 + 40*(\overline{q})_{x}*(q)_{x}*q - (q)_{xxxxxx} + 12*(q)_{xxxx}*\overline{q}*q - 10*(q)_{xxxx} + 30*(q)_{xxx}*(q)_{x}*\overline{q} + 20*(q)_{xx}**2*\overline{q} - 50*(q)_{xx}*\overline{q}**2*q**2 + 80*(q)_{xx}*\overline{q}*q - 30*(q)_{xx} - 70*(q)_{x}**2*\overline{q}**2*q + 60*(q)_{x}**2*\overline{q} + 20*\overline{q}**3*q**4 - 60*\overline{q}**2*q**3 + 60*\overline{q}*q**2 - 20*q)

In [ ]:
for n in KdV_hierarchy_plus_RHS.keys():
    display(Equ(multi_deriv(u, t_list[n]), subs(simplify(KdV_hierarchy_plus_RHS[n]), [(W_plus, u)])))

In [ ]:
for n in rGP_hierarchy_full_RHS.keys():
    display(Equ(rGP_hierarchy_full_LHS[n][0], simplify(rGP_hierarchy_full_RHS[n][0]).expand()))
    display(Equ(rGP_hierarchy_full_LHS[n][1], simplify(rGP_hierarchy_full_RHS[n][1]).expand()))

In [ ]:
for n in wpGP_hierarchy_full_RHS.keys():
    display(Equ(wpGP_hierarchy_full_LHS[n], Matrix([
        simplify(wpGP_hierarchy_full_RHS[n][0] + KdV_hierarchy_plus_RHS[n-1] + O(epsilon**2)), 
        simplify(wpGP_hierarchy_full_RHS[n][1] + O(epsilon**2))
    ])))
    display(Equ(wmGP_hierarchy_full_LHS[n], Matrix([
        simplify(wmGP_hierarchy_full_RHS[n][0] + O(epsilon**2)), 
        simplify(wmGP_hierarchy_full_RHS[n][1] - KdV_hierarchy_minus_RHS[n-1] + O(epsilon**2))
    ])))

In [ ]:
def order_epsilon(expr):
    from sympy import collect, Order
    if expr.has(Order):
        o = expr.getO()
        p = expr.removeO()
    else:
        o = 0
        p = expr
    p = collect(p, epsilon)
    terms = p.as_ordered_terms()
    sorted_terms = sorted(terms, key=lambda t: t.as_coeff_exponent(epsilon)[1])
    return sum(sorted_terms) + o


In [ ]:
for n in [0, 1, 2]:
    display(Equ(Symbol(f'(W_+)_{{T_{{{n}}}}}'), collect(simplify(wpGP_hierarchy_full_RHS[n][0] + wmGP_hierarchy_full_RHS[n][0] + O(epsilon**4)).expand(), epsilon)))
    display(Equ(Symbol(f'(W_-)_{{T_{{{n}}}}}'), collect(simplify(wpGP_hierarchy_full_RHS[n][1] + wmGP_hierarchy_full_RHS[n][1] + O(epsilon**4)).expand(), epsilon)))
for n in [0, 1, 2]:
    display(Equ(Symbol(f'(W_+)_{{T_{{{n}}}}}'), collect(simplify(wpGP_hierarchy_full_RHS[n][0] - wmGP_hierarchy_full_RHS[n][0] + O(epsilon**4)).expand(), epsilon)))
    display(Equ(Symbol(f'(W_-)_{{T_{{{n}}}}}'), collect(simplify(wpGP_hierarchy_full_RHS[n][1] - wmGP_hierarchy_full_RHS[n][1] + O(epsilon**4)).expand(), epsilon)))

for n in [0, 1, 2]:
    display(Equ(Symbol(f'(W_+)_{{T_{{{n}}}}}'), collect(simplify(wmGP_hierarchy_full_RHS[n][0] + O(epsilon**4)).expand(), epsilon)))
    display(Equ(Symbol(f'(W_-)_{{T_{{{n}}}}}'), collect(simplify(wmGP_hierarchy_full_RHS[n][1] + O(epsilon**4)).expand(), epsilon)))


# GP vs hGP Hierarchy

In [ ]:
subs_list_1 = [
    (w_minus, v / 2 - (a - 1)), 
    (w_plus, v / 2 + (a - 1)),
]
subs_list_2 = [
    (a, sqrt(q * q_conj)), 
    (v, (q_conj * deriv(q) - q * deriv(q_conj)) / (2 * I * q * q_conj))
]
for n in GP_hierarchy_full_RHS.keys():
    substituted_hGP_hierarchy_full_LHS = simplify(subs(simplify(subs(hGP_hierarchy_full_LHS[n], subs_list_1)), subs_list_2))
    substituted_hGP_hierarchy_full_RHS = simplify(subs(simplify(subs(hGP_hierarchy_full_RHS[n], subs_list_1)), subs_list_2))
    equation_applied_to_substituted_hGP_hierarchy_full_LHS = subs(substituted_hGP_hierarchy_full_LHS, [
        (GP_hierarchy_full_LHS[n][0], GP_hierarchy_full_RHS[n][0]), (GP_hierarchy_full_LHS[n][1], GP_hierarchy_full_RHS[n][1])
    ])
    display(Equ(Symbol(f'\\text{{Error}}_{{\\text{{GP vs hGP hierarchy}}}}^{{{n}}}'), simplify(substituted_hGP_hierarchy_full_RHS - equation_applied_to_substituted_hGP_hierarchy_full_LHS)))

# hGP vs rGP Hierarchy

In [ ]:
subs_list_2 = [
    (A, a),
    (W_minus, epsilon**(-2) * w_minus), 
    (W_plus, epsilon**(-2) * w_plus),
]
subs_scale = {
    t: 1, #epsilon**(-3) * sqrt(2), 
    x: epsilon**(-1) / sqrt(2)
}
for n in hGP_hierarchy_full_RHS.keys():
    substituted_rGP_hierarchy_full_LHS = simplify(subs(simplify(subs(rGP_hierarchy_full_LHS[n], subs_list_1)), subs_list_2, scale=subs_scale))
    substituted_rGP_hierarchy_full_RHS = simplify(subs(simplify(subs(rGP_hierarchy_full_RHS[n], subs_list_1)), subs_list_2, scale=subs_scale))
    equation_applied_to_substituted_rGP_hierarchy_full_LHS = subs(substituted_rGP_hierarchy_full_LHS, [
        (hGP_hierarchy_full_LHS[n][0], hGP_hierarchy_full_RHS[n][0]), (hGP_hierarchy_full_LHS[n][1], hGP_hierarchy_full_RHS[n][1])
    ])
    display(Equ(Symbol(f'\\text{{Error}}_{{\\text{{hGP vs rGP hierarchy}}}}^{{{n}}}'), simplify(substituted_rGP_hierarchy_full_RHS - equation_applied_to_substituted_rGP_hierarchy_full_LHS)))

# Approximation Iterations

In [ ]:
predicted_sig_expr_wpGP = {n: 0 for n in sig_expr_rGP.keys()}
predicted_sig_expr_wmGP = {n: 0 for n in sig_expr_rGP.keys()}
predicted_H_wpGP = {n: 0 for n in H_rGP.keys()}
predicted_H_wmGP = {n: 0 for n in H_rGP.keys()}

for n in range(0, N//2+1):
    X = re(sig_expr_rGP[2*n+1])
    Y = re(sigt_expr_rGP[2*n+1])
    for j in range(n+1):
        expansion_coeff = I * binomial(-Rational(1,2), n-j) * (I * sqrt(2) / epsilon)**(2*n-2*j+1)
        X += expansion_coeff * im(sig_expr_rGP[2*j])
        Y -= expansion_coeff * im(sigt_expr_rGP[2*j])
    predicted_sig_expr_wmGP[n] = collect(simplify(X).expand(), epsilon)
    predicted_sig_expr_wpGP[n] = collect(simplify(Y).expand(), epsilon)

    
# for n in range(0, N//2+1):
#     X = re(H_rGP[2*n])
#     Y = re(Ht_rGP[2*n])
#     for j in range(n+1):
#         expansion_coeff = I * binomial(-Rational(1,2), n-j) * (I * sqrt(2) / epsilon)**(2*n-2*j+1) / (- I**(2*n) / (sqrt(2) * epsilon)**(2*n+1))
#         X -= expansion_coeff * im(- I**(2*j-1) / (sqrt(2) * epsilon)**(2*j) * (2 - (1 if j == 0 else 0)) / 2 * H_rGP[2*j-1])
#         Y += expansion_coeff * im(- I**(2*j-1) / (sqrt(2) * epsilon)**(2*j) * (2 - (1 if j == 0 else 0)) / 2 * Ht_rGP[2*j-1])
#     X *= (- I**(2*n) / (sqrt(2) * epsilon)**(2*n+1))
#     Y *= (- I**(2*n) / (sqrt(2) * epsilon)**(2*n+1))
#     predicted_H_wmGP[n] = collect(simplify(X).expand(), epsilon)
#     predicted_H_wpGP[n] = collect(simplify(Y).expand(), epsilon)


for n in range(0, N//2+1):
    X = re(H_rGP[2*n])
    Y = re(Ht_rGP[2*n])
    for j in range(n+1):
        expansion_coeff = binomial(-Rational(1,2), n-j) * 4**(n-j) * (2 - (1 if j == 0 else 0))
        X += expansion_coeff * re(H_rGP[2*j-1])
        Y -= expansion_coeff * re(Ht_rGP[2*j-1])
    X *= (-1)**(n+1) / (sqrt(2) * epsilon)**(2*n+1)
    Y *= (-1)**(n+1) / (sqrt(2) * epsilon)**(2*n+1)
    predicted_H_wmGP[n] = collect(simplify(X).expand(), epsilon)
    predicted_H_wpGP[n] = collect(simplify(Y).expand(), epsilon)

for n in range(-1, N//2+1):
    display(Equ(Symbol(f'\\text{{Error}}_{{\\text{{wpGP sigma vs H}}}}^{{{n}}}'), simplify(predicted_H_wpGP[n] - predicted_sig_expr_wpGP[n])))
    display(Equ(Symbol(f'\\text{{Error}}_{{\\text{{wmpGP sigma vs H}}}}^{{{n}}}'), simplify(predicted_H_wmGP[n] - predicted_sig_expr_wmGP[n])))
    display(Equ(Symbol(f'\\text{{Error}}_{{\\text{{wpGP vs KdV}}}}^{{{n}}}'), collect(simplify(predicted_H_wpGP[n] - sig_eq_plus_KdV[2*n+1] + O(epsilon**2)).expand().removeO(), epsilon)))
    display(Equ(Symbol(f'\\text{{Error}}_{{\\text{{mpGP vs KdV}}}}^{{{n}}}'), collect(simplify(predicted_H_wmGP[n] - sig_eq_minus_KdV[2*n+1] + O(epsilon**2)).expand().removeO(), epsilon)))


In [ ]:
H_NLS.keys()

In [ ]:
# NLS_hierarchy_q_LHS = {n: I * multi_deriv(q, t) for n in H_NLS.keys()}
# NLS_hierarchy_q_RHS = {n: variation(H_NLS[n], q_conj) for n in H_NLS.keys()}
# GP_hierarchy_q_LHS = {n: I * multi_deriv(q, t) for n in H_GP.keys()}
# GP_hierarchy_q_RHS = {n: variation(H_GP[n], q_conj) for n in H_GP.keys()}

NLS_hierarchy_q_LHS = {n: I * NLS_hierarchy_full_LHS[n][0] for n in NLS_hierarchy_full_LHS.keys()}
NLS_hierarchy_q_RHS = {n: I * NLS_hierarchy_full_RHS[n][0] for n in NLS_hierarchy_full_RHS.keys()}
GP_hierarchy_q_LHS = {n: I * GP_hierarchy_full_LHS[n][0] for n in GP_hierarchy_full_LHS.keys()}
GP_hierarchy_q_RHS = {n: I * GP_hierarchy_full_RHS[n][0] for n in GP_hierarchy_full_RHS.keys()}

# Expansions - NLS vs GP

In [ ]:
display(tsimplify(subs(sig_eqn_lam_NLS, [(sig, sig + I * lam)]).expand()))
display(tsimplify(subs(sig_eqn_GP, [(sig, sig + I * z), (z, z_GP)]).expand()))

sig_expr_GP_from_NLS = {n: 0 for n in sig_expr_NLS.keys()}
sig_expr_NLS_from_GP = {n: 0 for n in sig_expr_GP.keys()}

for n in sig_expr_NLS.keys():
    nn = n // 2
    if n % 2 == 0:
        sig_expr_GP_from_NLS[2*nn] = 0
        for m in range(nn+1):
            sig_expr_GP_from_NLS[2*nn] += binomial(nn-1, nn-m) * 4**(nn-m) * sig_expr_NLS[2*m]
        sig_expr_NLS_from_GP[2*nn] = 0
        for m in range(nn+1):
            sig_expr_NLS_from_GP[2*nn] += binomial(nn-1, nn-m) * (-4)**(nn-m) * sig_expr_GP[2*m]
    else:
        sig_expr_GP_from_NLS[2*nn+1] = 0
        for m in range(nn+1):
            sig_expr_GP_from_NLS[2*nn+1] += binomial(nn-Rational(1,2), nn-m) * 4**(nn-m) * sig_expr_NLS[2*m+1]
        sig_expr_GP_from_NLS[2*nn+1] += catalan(nn)
        sig_expr_NLS_from_GP[2*nn+1] = 0
        for m in range(nn+1):
            sig_expr_NLS_from_GP[2*nn+1] += binomial(nn-Rational(1,2), nn-m) * (-4)**(nn-m) * sig_expr_GP[2*m+1]
        sig_expr_NLS_from_GP[2*nn+1] += (-1)**(nn+1) * catalan(nn)

error = 0
for n in sig_expr_NLS.keys():
    error += simplify(sig_expr_NLS[n] - sig_expr_NLS_from_GP[n])**2
display(Equ(Symbol('\\text{Error}_{\\text{NLS from GP $\\sigma$}}'), error))
error = 0
for n in sig_expr_NLS.keys():
    error += simplify(sig_expr_GP[n] - sig_expr_GP_from_NLS[n])**2
display(Equ(Symbol('\\text{Error}_{\\text{GP from NLS $\\sigma$}}'), error))


H_GP_from_NLS = {n: 0 for n in H_NLS.keys()}
H_NLS_from_GP = {n: 0 for n in H_GP.keys()}

for n in [k for k in sig_expr_NLS.keys() if k >= 0]:
    nn = n // 2
    if n % 2 == 0:
        H_GP_from_NLS[2*nn-1] = 0
        for m in range(1, nn+1):
            H_GP_from_NLS[2*nn-1] += binomial(nn-1, nn-m) * (-4)**(nn-m) * H_NLS[2*m-1]
        H_NLS_from_GP[2*nn-1] = 0
        for m in range(1, nn+1):
            H_NLS_from_GP[2*nn-1] += binomial(nn-1, nn-m) * 4**(nn-m) * H_GP[2*m-1]
    else:
        H_GP_from_NLS[2*nn] = 0
        for m in range(nn+1):
            H_GP_from_NLS[2*nn] += binomial(nn-Rational(1,2), nn-m) * (-4)**(nn-m) * H_NLS[2*m]
        H_GP_from_NLS[2*nn] += - I**(2*nn) * catalan(nn)
        H_NLS_from_GP[2*nn] = 0
        for m in range(nn+1):
            H_NLS_from_GP[2*nn] += binomial(nn-Rational(1,2), nn-m) * 4**(nn-m) * H_GP[2*m]
        H_NLS_from_GP[2*nn] += catalan(nn)
error = 0
for n in [k for k in H_NLS.keys() if k >= 0]:
    error += simplify(H_NLS[n] - H_NLS_from_GP[n])**2
display(Equ(Symbol('\\text{Error}_{\\text{NLS from GP $H$}}'), error))
error = 0
for n in [k for k in H_NLS.keys() if k >= 0]:
    error += simplify(H_GP[n] - H_GP_from_NLS[n])**2
display(Equ(Symbol('\\text{Error}_{\\text{GP from NLS $H$}}'), error))


hierarchy_GP_from_NLS = {n: 0 for n in NLS_hierarchy_q_RHS.keys()}
hierarchy_NLS_from_GP = {n: 0 for n in GP_hierarchy_q_RHS.keys()}

for n in [k for k in sig_expr_NLS.keys() if k >= 0]:
    nn = n // 2
    if n % 2 == 0:
        hierarchy_GP_from_NLS[2*nn-1] = 0
        for m in range(1, nn+1):
            hierarchy_GP_from_NLS[2*nn-1] += binomial(nn-1, nn-m) * (-4)**(nn-m) * NLS_hierarchy_q_RHS[2*m-1]
        hierarchy_NLS_from_GP[2*nn-1] = 0
        for m in range(1, nn+1):
            hierarchy_NLS_from_GP[2*nn-1] += binomial(nn-1, nn-m) * 4**(nn-m) * GP_hierarchy_q_RHS[2*m-1]
    else:
        hierarchy_GP_from_NLS[2*nn] = 0
        for m in range(nn+1):
            hierarchy_GP_from_NLS[2*nn] += binomial(nn-Rational(1,2), nn-m) * (-4)**(nn-m) * NLS_hierarchy_q_RHS[2*m]
        hierarchy_NLS_from_GP[2*nn] = 0
        for m in range(nn+1):
            hierarchy_NLS_from_GP[2*nn] += binomial(nn-Rational(1,2), nn-m) * 4**(nn-m) * GP_hierarchy_q_RHS[2*m]
error = 0
for n in [k for k in NLS_hierarchy_q_RHS.keys() if k >= 0]:
    error += simplify(NLS_hierarchy_q_RHS[n] - hierarchy_NLS_from_GP[n])**2
display(Equ(Symbol('\\text{Error}_{\\text{NLS from GP hierarchy}}'), error))
error = 0
for n in [k for k in GP_hierarchy_q_RHS.keys() if k >= 0]:
    error += simplify(GP_hierarchy_q_RHS[n] - hierarchy_GP_from_NLS[n])**2
display(Equ(Symbol('\\text{Error}_{\\text{GP from NLS hierarchy}}'), error))


# Modified linear part - NLS

In [ ]:
N_old = N
N = 20

def A_coeff(n, j):
    return - 4**j * binomial(Rational(n,2) - 1, j)
error = 0
for n in range(2, N+1):
    if n == 2:
        X = - A_coeff(2, 0) - 1
        error += X**2
    else:
        for j in range((n+1)//2):
            X = - A_coeff(n+1, j)
            for k in range(1, 2*j, 2):
                X += 2 * catalan(Rational(k-1,2)) * A_coeff(n-k, j-Rational(k+1,2))
            if j == Rational(n-1,2):
                X += - catalan(Rational(n-1,2)) * Rational(n+1,2)
            else:
                X += A_coeff(n, j)
            error += X**2
display(Equ(Symbol('\\text{Error}_{\\text{$A$ formula vs iteration}}'), error))


def B_coeff(n, j):
    return - sum([(-1)**l * (catalan(l+1) - 2*catalan(l)) * 4**(j-l) * binomial(Rational(n-1,2) - 1, j - l) for l in range(j+1)])
error = 0
for n in range(4, N+1):
    if n == 4:
        X = - B_coeff(2, 0) + 1
        error += X**2
    else:
        for j in range((n+1)//2-1):
            X = - B_coeff(n+1, j)
            for k in range(1, 2*j, 2):
                X += 2 * catalan(Rational(k-1,2)) * B_coeff(n-k, j-Rational(k+1,2))
            if j == Rational(n-3,2):
                X += catalan(Rational(n-1,2)) * Rational(n-1,2)
            else:
                X += B_coeff(n, j)
            error += X**2
display(Equ(Symbol('\\text{Error}_{\\text{$B$ formula vs iteration}}'), error))
     

def D_coeff(n, j):
    return - A_coeff(n, j)


def E_coeff(n, j):
    return - B_coeff(n, j)


def Ft_coeff(n, j):
    m = n // 2
    if n % 2 == 0:
        return 0
    if n % 2 == 1:
        return - (8*m-8*j-6) * 4**j * binomial(m-Rational(1,2), j)
error = 0
for n in range(1, N+1):
    for j in range((n-1)//2-1):
        X = - Ft_coeff(n+1, j) 
        if n % 2 == 0:
            X += 2 * sum([Ft_coeff(n-2*k-1, j-k-1) * catalan(k) for k in range(j)]) - 2 * (j+1) * D_coeff(n, j+1)
            X += 2 * sum([sum([(-1)**t * (1 + (1 if t != Rational(n,2)-1-j else 0)) * D_coeff(t+1+2*k, k) * D_coeff(n-1-2*k-t, j-k) for t in range(1, n//2-j)]) for k in range(j+1)])
        error += X**2
display(Equ(Symbol('\\text{Error}_{\\text{$\\tilde{F}$ formula vs iteration}}'), error))
error = 0
for n in range(1, N+1):
    for j in range((n-1)//2-1):
        X = - Ft_coeff(n+1, j) 
        if n % 2 == 0:
            X += 2 * sum([Ft_coeff(n-2*k-1, j-k-1) * catalan(k) for k in range(j)]) - 2 * (-4)**j * binomial(j-n//2, j) - 2 * (j+1) * 4**(j+1) * binomial(n//2-1, j+1)
        error += X**2
display(Equ(Symbol('\\text{Error}_{\\text{$\\tilde{F}$ formula vs substituted iteration}}'), error))


def Gt_coeff(n, j):
    X = (-1)**(n+1) * 4 * ((n-1)//2-j) * sum([catalan(k) * 4**(j-1-k) * binomial((n-1)//2-Rational(1,2)-k, j-1-k) for k in range(j)])
    if n % 2 == 1:
        X += 2 * 4**(j-1) * binomial(Rational(n,2)-1, j-1)
    return X
error = 0
for n in range(1, N+1):
    for j in range((n-1)//2):
        X = - Gt_coeff(n+1, j) + 2 * sum([Gt_coeff(n-2*k-1, j-k-1) * catalan(k) for k in range(j)]) - (j+1) * E_coeff(n, j-1) + (-1)**n * j * D_coeff(n, j)
        X += 2 * sum([sum([(-1)**t * D_coeff(t+1+2*k, k) * E_coeff(n-1-2*k-t, j-2-k) for t in range(1, n-2*j)]) for k in range(0, j-1)])
        error += X**2
display(Equ(Symbol('\\text{Error}_{\\text{$\\tilde{G}$ formula vs iteration}}'), error))
error = 0
for n in range(1, N+1):
    for j in range((n-1)//2):
        X = - Gt_coeff(n+1, j) + 2 * sum([Gt_coeff(n-2*k-1, j-k-1) * catalan(k) for k in range(j)]) + (-1)**n * j * 4**j * binomial(Rational(n,2)-1, j)
        X += - sum([(catalan(l+1) - 2*catalan(l)) * (-1)**l * (
            (j+1) * 4**(j-1-l) * binomial(Rational(n-1,2)-1, j-1-l)
            + 2* (1 if n%2==0 else 0) * 4**(j-2-l) * binomial(Rational(n-1,2)-1, j-2-l)
        ) for l in range(j)])
        error += X**2
display(Equ(Symbol('\\text{Error}_{\\text{$\\tilde{G}$ formula vs substituted iteration}}'), error))
      
    
def J_coeff_unrefined(n, j):
    return (1 if j<=(n-1)//2-2 else 0) * Ft_coeff(n, j) + (1 if n%2==1 else 0) * 2 * (j+1) * D_coeff(n, j+1)
def J_coeff(n, j):
    return 2 * 4**j * binomial(Rational(n,2)-1, j) if n%2==1 else 0
error = 0
for n in range(N+1):
    for j in range(n//2-1):
        error += simplify(J_coeff_unrefined(n, j) - J_coeff(n, j))**2
display(Equ(Symbol('\\text{Error}_{\\text{$J$ different formula}}'), error))


def K_coeff_unrefined(n, j):
    return  (1 if j<=(n-1)//2-1 else 0) * Gt_coeff(n, j) + (j+1) * ((1 if j>=1 else 0) * E_coeff(n, j-1) + (-1)**n * D_coeff(n, j))
def K_coeff_half_refined(n, j):
    return (-1)**n * D_coeff(n+1, j) + 2 * (1 if n%2==1 else 0) * E_coeff(n, j-2)
def K_coeff(n, j):
    return (-1)**n * 4**j * binomial(Rational(n-1,2), j) + 2 * (1 if n%2==1 else 0) * sum([(-1)**l * (catalan(l+1) - 2*catalan(l)) * 4**(j-2-l) * binomial(Rational(n-1,2) - 1, j-2-l) for l in range(j-1)])
error = 0
for n in range(N+1):
    for j in range(n//2):
        error += simplify(K_coeff_unrefined(n, j) - K_coeff_half_refined(n, j))**2
for n in range(N+1):
    for j in range(n//2):
        error += simplify(K_coeff_half_refined(n, j) - K_coeff(n, j))**2
display(Equ(Symbol('\\text{Error}_{\\text{$K$ different formula}}'), error))

N = N_old

In [ ]:
def pi_0_r_NLS(n):
    return 0 if (n%2==0) else - catalan(Rational(n-1,2)) * q_conj**Rational(n+1,2) * (-q)**Rational(n-1,2)
error = 0
for n in range(N+1):
    error += simplify(pi_0_r_NLS(n) - extract_deriv(r_expr_NLS[n], 0))**2
    
display(Equ(Symbol('\\text{Error}_{\\text{NLS $\pi_0 r$ formula vs iteration}}'), error))


def pi_0_sig_NLS(n):
    return 0 if (n%2==0) else catalan(Rational(n-1,2)) * q_conj**Rational(n+1,2) * (-q)**Rational(n+1,2)
error = 0
for n in range(N+1):
    error += simplify(pi_0_sig_NLS(n) - extract_deriv(sig_expr_NLS[n], 0))**2
display(Equ(Symbol('\\text{Error}_{\\text{NLS $\pi_0 \\sigma$ formula vs iteration}}'), error))


def pi_1_r_NLS(n):
    X = 0
    for j in range(n//2):
        X += A_coeff(n, j) * (-q)**j * q_conj**j * higher_deriv(q_conj, n-1-2*j)
    for j in range(n//2 - 1):
        X += B_coeff(n, j) * (-q)**j * q_conj**(j+2) * higher_deriv(q, n-3-2*j)
    return X
error = 0
for n in range(N+1):
    error += simplify(pi_1_r_NLS(n) - extract_deriv(r_expr_NLS[n], 1))**2
display(Equ(Symbol('\\text{Error}_{\\text{NLS $\pi_1 r$ formula vs iteration}}'), error))


def pi_1_sig_NLS(n):
    X = 0
    for j in range(n//2):
        X += D_coeff(n, j) * (-q)**(j+1) * q_conj**j * higher_deriv(q_conj, n-1-2*j)
    for j in range(n//2 - 1):
        X += E_coeff(n, j) * (-q)**(j+1) * q_conj**(j+2) * higher_deriv(q, n-3-2*j)
    return X
error = 0
for n in range(N+1):
    error += simplify(pi_1_sig_NLS(n) - extract_deriv(sig_expr_NLS[n], 1))**2
display(Equ(Symbol('\\text{Error}_{\\text{NLS $\pi_1 \\sigma$ formula vs iteration}}'), error))

def pi_1_delta_pi_2_alt_sig_NLS(n):
    X = 0
    for j in range((n-1)//2 - 1):
        X += Ft_coeff(n, j) * (-q)**(j+2) * q_conj**j * higher_deriv(q_conj, n-3-2*j)
    for j in range((n-1)//2):
        X += Gt_coeff(n, j) * (-q)**j * q_conj**j * higher_deriv(q, n-1-2*j)
    return X
error = 0
for n in range(N+1):
    error += simplify(pi_1_delta_pi_2_alt_sig_NLS(n) - extract_deriv(variation(extract_deriv_alt(sig_expr_NLS[n], 2), q_conj), 1))**2
display(Equ(Symbol('\\text{Error}_{\\text{NLS $\pi_1 \delta \\tilde{\pi_2} \\sigma$ formula vs iteration}}'), error))

def pi_1_delta_sig_NLS(n):
    X = 0
    for j in range(n//2-1):
        X += J_coeff(n, j) * (-q)**(j+2) * q_conj**j * higher_deriv(q_conj, n-3-2*j)
    for j in range(n//2):
        X += K_coeff(n, j) * (-q)**j * q_conj**j * higher_deriv(q, n-1-2*j)
    return X
error = 0
for n in range(N+1):
    error += simplify(pi_1_delta_sig_NLS(n) - extract_deriv(variation(sig_expr_NLS[n], q_conj), 1))**2
display(Equ(Symbol('\\text{Error}_{\\text{NLS $\pi_1 \delta \\sigma$ formula vs iteration}}'), error))


# Structure of the NLS and GP hierarchies

In [ ]:
print(NLS_hierarchy_q_RHS.keys())

In [ ]:
error = 0
for n in range(N):
    NLS_hierarchy_q_RHS_predicted = 0
    m = n // 2
    if n % 2 == 0:
        NLS_hierarchy_q_RHS_predicted += - (-1)**m * sum([J_coeff(2*m+1, j) * (-q)**(j+2) * q_conj**j * higher_deriv(q_conj, 2*m-2-2*j) for j in range(m-1)])
        NLS_hierarchy_q_RHS_predicted += - (-1)**m * sum([K_coeff(2*m+1, j) * (-q)**j * q_conj**j * higher_deriv(q, 2*m-2*j) for j in range(m)])
        NLS_hierarchy_q_RHS_predicted += - (-1)**m * (m+1) * catalan(m) * (-q)**(m+1) * q_conj**m
        error += simplify(extract_deriv(simplify(NLS_hierarchy_q_RHS[2*m] - NLS_hierarchy_q_RHS_predicted), 1))**2
    if n % 2 == 1:
        NLS_hierarchy_q_RHS_predicted += I * (-1)**m * sum([K_coeff(2*m+2, j) * (-q)**j * q_conj**j * higher_deriv(q, 2*m+1-2*j) for j in range(m+1)])
        error += simplify(extract_deriv(simplify(NLS_hierarchy_q_RHS[2*m+1] - NLS_hierarchy_q_RHS_predicted), 1))**2
display(Equ(Symbol('\\text{Error}_{\\text{NLS hierarchy predicted part error}}'), error))

error = 0
for n in range(N):
    GP_hierarchy_q_RHS_predicted = 0
    m = n // 2
    if n % 2 == 0:
        for k in range(m+1):
            X = - sum([(-1)**(k-j) * J_coeff(2*k+1, j) * q**(j+2) * q_conj**j * higher_deriv(q_conj, 2*k-2-2*j) for j in range(k-1)])
            X += - sum([(-1)**(k-j) * K_coeff(2*k+1, j) * q**j * q_conj**j * higher_deriv(q, 2*k-2*j) for j in range(k)])
            X += (k+1) * catalan(k) * q**(k+1) * q_conj**k
            X *= binomial(m - Rational(1,2), m - k) * (-4)**(m-k)
            GP_hierarchy_q_RHS_predicted += X
        error += simplify(extract_deriv(simplify(GP_hierarchy_q_RHS[2*m] - GP_hierarchy_q_RHS_predicted), 1))**2
    if n % 2 == 1:
        for k in range(m+1):
            X = - I * sum([(-1)**(k-j) * K_coeff(2*k, j) * q**j * q_conj**j * higher_deriv(q, 2*k-1-2*j) for j in range(k)])
            X *= binomial(m - 1, m - k) * (-4)**(m-k)
            GP_hierarchy_q_RHS_predicted += X
        error += simplify(extract_deriv(simplify(GP_hierarchy_q_RHS[2*m-1] - GP_hierarchy_q_RHS_predicted), 1))**2
display(Equ(Symbol('\\text{Error}_{\\text{GP hierarchy predicted part error}}'), error))


# Modified linear part - GP

In [ ]:
for n in  GP_modified_linear_part.keys():
    print("n =", n)
    display(GP_hierarchy_q_RHS[n])
    display(GP_modified_linear_part[n])

In [ ]:
error1 = 0
error2 = 0

GP_modified_linear_part = {n: 0 for n in H_GP.keys()}
for n in GP_modified_linear_part.keys():
    GP_modified_linear_part[n] = subs(extract_deriv(variation(H_GP[n], q_conj), 1), [(q, p), (q_conj, p_conj)])
    if hasattr(GP_modified_linear_part[n], 'subs'):
        GP_modified_linear_part[n] = simplify(simplify(GP_modified_linear_part[n].subs(p, 1 / p_conj)).subs(p_conj, 1 / q_pm))
    
for n in H_GP.keys():
    m = (n+1)//2
    
    if n % 2 == 0:
        GP_modified_linear_part_first = q_pm**2 * sum([
            sum([
                (I**(2*j) * higher_deriv(p_conj, 2*j)) * binomial(m-Rational(1,2), m-k) * (-4)**(m-k) * J_coeff(2*k+1, k-1-j) 
            for k in range(j+1, m+1)]) 
        for j in range(1, m)])
        GP_modified_linear_part_first += - sum([
            sum([
                (I**(2*j) * higher_deriv(p, 2*j)) * binomial(m-Rational(1,2), m-k) * (-4)**(m-k) * K_coeff(2*k+1, k-j) 
            for k in range(j, m+1)]) 
        for j in range(1, m+1)])
        
        GP_modified_linear_part_second = (1 if m >= 2 else 0) * q_pm**2 * 2 * I**(2*m-2) * higher_deriv(p_conj, 2*m-2)
        GP_modified_linear_part_second += (1 if m >= 2 else 0) * 2*I**(2*m-2) * higher_deriv(p, 2*m-2)
        GP_modified_linear_part_second += (1 if m >= 1 else 0) * I**(2*m) * higher_deriv(p, 2*m)
    
    if n % 2 == 1:
        GP_modified_linear_part_first = sum([
            sum([
                (I**(2*j+1) * higher_deriv(p, 2*j+1)) * binomial(m-1, m-k) * (-4)**(m-k) * K_coeff(2*k, k-1-j) 
            for k in range(j+1, m+1)]) 
        for j in range(0, m)])
    
        GP_modified_linear_part_second = sum([4**(m-1-j) * binomial(Rational(1,2), m-1-j) * I**(2*j+1) * higher_deriv(p, 2*j+1) for j in range(m)])
        
    error1 += simplify(GP_modified_linear_part_first - GP_modified_linear_part[n])
    error2 += simplify(GP_modified_linear_part_second - GP_modified_linear_part[n])
display(Equ(Symbol('\\text{Error}_{\\text{GP modified linear part first formula error}}'), error1))
display(Equ(Symbol('\\text{Error}_{\\text{GP modified linear part second formula error}}'), error2))


In [ ]:
for n in GP_modified_linear_part.keys():
    m = (n+1)//2
    if n % 2 == 1:
        print("n =", n, "m =", m)
        X = simplify(sum([binomial(-Rational(1,2), m-k) * 4**(m-k) * GP_modified_linear_part[2*k-1] for k in range(m+1)]))
        display(X)
        

In [ ]:
X, Y = symbols('X Y')

expr = Rational(1,2) * (1 - 1/sqrt(1+4*Y)) * (-8*Y/sqrt(1+4*Y)**3 + 4/sqrt(1+4*Y))
expr += Y/sqrt(1+4*Y) * (-16*Y/sqrt(1+4*Y)**3 + 4/sqrt(1+4*Y))

expr3 = 1 / sqrt(1+4*Y) + 1 / sqrt(1+4*Y)**3 - 2/(1+4*Y)**2
display(simplify(expr - expr3))

ser = series_expansion(expr3, 0, Y, 20)
ser[0] = 0

for j in range(0, 20):
    print(ser[j] - Gt_coeff(0, j))


In [ ]:
expr = Matrix([[u + f, g], [- g, u - f]])
diag = expr.diagonalize()
display(diag[0])
display(diag[1])
display(simplify(diag[0].inv() * diag[1] * diag[0]))
display(simplify(diag[0] * diag[1] * diag[0].inv()))

In [ ]:

for n in range(N+1):
    display(Equ(sig_sym[n], simplify(subs(sig_eq[n], [(u, W_minus), (v, W_plus), (epsiloninv, 1 / epsilon)]) - sig_iter_evaluated_rGP[n])))
    display(Equ(r_sym[n], simplify(subs(- I * r_eq[n], [(u, W_minus), (v, W_plus), (epsiloninv, 1 / epsilon)]) - r_iter_evaluated_rGP[n])))

In [ ]:


def q_coeff(n):
    if n == -1:
        return epsilon / (I * sqrt(2))
    if n < -1 or (n % 2 == 0 and n >= 0):
        return 0
    else:
        return I * catalan((n - 1) // 2) / sqrt(2)**n * epsiloninv**n
       
def q_coeff_plus(n):
    if n == 0:
        return 1
    return q_coeff(n)

def q_coeff_minus(n):
    if n == 0:
        return 1
    return - q_coeff(n)
    
def qinv_coeff(n):
    if n < 0 or n % 2 == 0:
        return 0
    return binomial(- Rational(1,2), (n-1)//2) * sqrt(2)**n * I**n * epsiloninv**n

def sqrtt(x):
    return I*sqrt(-x)


X = Symbol('X')

def get_expansion(eqn, x0, var, num):
    epsilonpos = Symbol('epsilon_2', positive=True)
    epsiloninvpos = Symbol('eta_2', positive=True)
    
    eqn = eqn.subs(epsilon, epsilonpos).subs(epsiloninv, epsiloninvpos)
    ser = series(eqn, x0=0, x=var, n=num).removeO().subs(epsilonpos, epsilon).subs(epsiloninvpos, epsiloninv)
    return [simplify(ser.coeff(var**n) if n > 0 else ser.subs(var, 0)) for n in range(0, num)]
    
N = 8

h_plus = get_expansion(1 + 1 / sqrtt(1 - Rational(1, 2) / epsiloninv**2 / X**2), 0, X, N+1)
h_minus = get_expansion(1 - 1 / sqrtt(1 - Rational(1, 2) / epsiloninv**2 / X**2), 0, X, N+1)

    
#h_plus = [binomial(-Rational(1, 2), n//2) * (sqrt(2) * I * epsiloninv)**n if n % 2 == 1 else 0 for n in range(N+1)]
#h_plus[0] = 1
#h_minus = [- binomial(-Rational(1, 2), n//2) * (sqrt(2) * I * epsiloninv)**n if n % 2 == 1 else 0 for n in range(N+1)]
#h_minus[0] = 1

h_plus_mod = get_expansion(1 + 1 / sqrtt(1 + Rational(1, 2) / epsiloninv**2 / X**2), 0, X, N+1)
h_minus_mod = get_expansion(1 - 1 / sqrtt(1 + Rational(1, 2) / epsiloninv**2 / X**2), 0, X, N+1)

exp1 = [get_expansion((X / sqrtt(1 - 1 / (8 * epsiloninv**2 * X**2)))**n, 0, X, N+1) for n in range(N+1)]

E_plus_sym = [Symbol('E^+_{' + str(n) + '}') for n in range(N+1)]
E_plus_eq = [E_plus_sym[n] for n in range(N+1)]

E_minus_sym = [Symbol('E^-_{' + str(n) + '}') for n in range(N+1)]
E_minus_eq = [E_plus_sym[n] for n in range(N+1)]

F_plus_sym = [Symbol('F^+_{' + str(n) + '}') for n in range(N+1)]
F_plus_eq = [E_plus_sym[n] for n in range(N+1)]

F_minus_sym = [Symbol('F^-_{' + str(n) + '}') for n in range(N+1)]
F_minus_eq = [E_plus_sym[n] for n in range(N+1)]

Ft_plus_sym = [Symbol('\\tilde{F}^+_{' + str(n) + '}') for n in range(N+1)]
Ft_plus_eq = [E_plus_sym[n] for n in range(N+1)]

Ft_minus_sym = [Symbol('\\tilde{F}^-_{' + str(n) + '}') for n in range(N+1)]
Ft_minus_eq = [E_plus_sym[n] for n in range(N+1)]

G_sym = [Symbol('G_{' + str(n) + '}') for n in range(N+1)]
G_eq = [G_sym[n] for n in range(N+1)]

r_factor = 1 #Symbol('lambda', real=True) #1

r_sym = [Symbol('r_{' + str(n) + '}') for n in range(N+1)]
r_eq = [r_sym[n] for n in range(N+1)]
r_poly = [r_sym[n] for n in range(N+1)]

r_eq[0] = r_factor * I
r_eq[1] = r_factor * epsilon / sqrt(2) * (v - u + 2 * epsiloninv**2)
r_poly[0] = polynomize(r_eq[0])
r_poly[1] = polynomize(r_eq[1])

sig_sym = [Symbol('\sigma_{' + str(n) + '}') for n in range(N+1)]
sig_eq = [sig_sym[n] for n in range(N+1)]
sig_poly = [sig_sym[n] for n in range(N+1)]

sig_eq[0] = q_coeff(-1) / 2 * (v + u)
sig_eq[1] = q_coeff(-1) / 2 * deriv(v + u) + q_coeff(-1)**2 / 4 * (v + u)**2 
sig_eq[1] += epsilon**2 / 2 * u * v + (u - v) / 2 
sig_eq[1] += epsilon / sqrt(2) * deriv(v) * r_eq[0]  

sig_poly[0] = polynomize(sig_eq[0])
sig_poly[1] = polynomize(sig_eq[1])

sig_plus_sym = [Symbol('\sigma_+^{' + str(n) + '}') for n in range(N+1)]
sig_plus_eq = [sig_plus_sym[n] for n in range(N+1)]

sig_minus_sym = [Symbol('\sigma_-^{' + str(n) + '}') for n in range(N+1)]
sig_minus_eq = [sig_minus_sym[n] for n in range(N+1)]

sig_plus_eq[0] = 0
sig_plus_eq[1] = - deriv(sig_plus_eq[0]) + sig_plus_eq[0]**2 - v

sig_minus_eq[0] = 0
sig_minus_eq[1] = deriv(sig_minus_eq[0]) + sig_minus_eq[0]**2 + u

for n in range(1, N):
    print("n =", n)

    X = - deriv(sig_plus_eq[n])
    
    for k in range(n+1):
        X += sig_plus_eq[k] * sig_plus_eq[n-k]
        
    sig_plus_eq[n+1] = simplify(X)
        
    X = deriv(sig_minus_eq[n])

    for k in range(n+1):
        X += sig_minus_eq[k] * sig_minus_eq[n-k]

    sig_minus_eq[n+1] = simplify(X)
        
    X = 0
    
    X += deriv(r_eq[n])

    X += - q_coeff(n) / sqrt(2) * epsiloninv
    
    for k in range(0, n+1):
        X += epsilon / sqrt(2) * (- v - epsiloninv**2) * r_eq[n-k] * r_eq[k]

    for m in range(0, n+1):
        for k in range(0, m+1):     
            X += - epsiloninv / sqrt(2) * q_coeff(n-m) * r_eq[m-k] * r_eq[k]

    for k in range(1, n+1):
        X += - q_coeff(-1) / sqrt(2) * epsiloninv * r_eq[n+1-k] * r_eq[k]

    r_eq[n+1] = r_factor * X
    r_poly[n+1] = polynomize(r_eq[n+1])
    r_eq[n+1] = depolynomize(r_poly[n+1])
    
    X = 0
    
    X += deriv(sig_eq[n])
    
    for k in range(0, n+1):
        X += sig_eq[n-k] * sig_eq[k]
        
    X += q_coeff(n) / 2 * (u + v)
    
    X += epsilon / sqrt(2) * deriv(v) * r_eq[n]
    
    sig_eq[n+1] = X
    sig_poly[n+1] = polynomize(sig_eq[n+1])
    sig_eq[n+1] = depolynomize(sig_poly[n+1])

for n in range(0, N+1):
    print("n =", n)
    E_plus_eq[n] = 0
    for k in range(0, n+1):
        E_plus_eq[n] += re(sig_eq[k] * h_plus[n-k])
        #E_plus_eq[n] += re(epsilon * r_eq[k] * h_plus[n-k])
        #E_plus_eq[n] += re(sig_eq[k] * q_coeff_plus(n-k))
    E_minus_eq[n] = 0
    for k in range(0, n+1):
        E_minus_eq[n] += re(sig_eq[k] * h_minus[n-k])
        #E_minus_eq[n] += re(epsilon * r_eq[k] * h_minus[n-k])
        #E_minus_eq[n] += re(sig_eq[k] * q_coeff_minus(n-k))
        
    F_plus_eq[n] = 0
    for k in range(0, n+1):
        F_plus_eq[n] += re(epsilon * r_eq[k] * h_plus[n-k])
        
    F_minus_eq[n] = 0
    for k in range(0, n+1):
        F_minus_eq[n] += re(epsilon * r_eq[k] * h_minus[n-k])
        
    G_eq[n] = 0
    for k in range(0, n+1):
        G_eq[n] += im(sig_eq[k]) * qinv_coeff(n-k) / I


In [ ]:
for n in range(N+1):
#     display(Equ(E_plus_sym[n], collect(simplify(E_plus_eq[n].subs(epsiloninv, epsilon)).expand(), epsilon)))
#     display(Equ(E_minus_sym[n], collect(simplify(E_minus_eq[n].subs(epsiloninv, epsilon)).expand(), epsilon)))
#     display(Equ(Y_plus_sym[n], collect(simplify(Y_plus_eq[n].subs(epsiloninv, epsilon)).expand(), epsilon)))
#     display(Equ(Y_minus_sym[n], collect(simplify(Y_minus_eq[n].subs(epsiloninv, epsilon)).expand(), epsilon)))
    display(Equ(EE_sym[n], collect(simplify(EE_eq[n].subs(epsiloninv, epsilon)).expand(), epsilon)))
    display(Equ(PP_sym[n], collect(simplify(PP_eq[n].subs(epsiloninv, epsilon)).expand(), epsilon)))